### 1-1. 데이터 복사하기

In [ ]:
from pathlib import Path
import shutil

def copy_folders(
    src_list,
    dst_root,
    overwrite=True,
    verbose=True,
):
    """
    여러 폴더(src_list)를 목적지(dst_root) 아래로 복사하는 범용 함수.

    Parameters
    ----------
    src_list : list[Path or str]
        복사할 폴더 목록
    dst_root : Path or str
        복사될 루트 디렉토리
    overwrite : bool
        목적지 폴더가 이미 있어도 덮어쓸지 여부
    verbose : bool
        복사 과정 출력 여부
    """

    dst_root = Path(dst_root)
    dst_root.mkdir(parents=True, exist_ok=True)

    dirs_exist_ok = overwrite

    for src in src_list:
        src = Path(src)

        if not src.exists():
            print(f"[WARN] 원본 폴더 없음: {src}")
            continue

        target = dst_root / src.name

        if verbose:
            print(f"📁 복사: {src} → {target}")

        shutil.copytree(src, target, dirs_exist_ok=dirs_exist_ok)

    print(f"\n[완료] 총 {len(src_list)}개 중 존재하는 폴더만 {dst_root} 아래로 복사했습니다.")


# ================================
# 사용 예시
# ================================
SRC_LIST = [
    "/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_good_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data",
]

DST = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data"

copy_folders(SRC_LIST, DST)

### 1-2. 데이터 1차 점검

In [ ]:
from pathlib import Path

def check_label_matching(
    base_dir: Path,
    image_ext: str = "*.jpg",
    label_ext: str = "*.txt",
    ignore_labels=("classes.txt",),
    verbose=True
):
    """
    이미지-라벨 매칭 여부 확인 유틸리티.
    하위 폴더별로 jpg ↔ txt 파일이 제대로 짝이 맞는지 검사한다.

    Parameters
    ----------
    base_dir : Path
        기준 폴더 (하위 폴더들까지 검사)
    image_ext : str
        이미지 확장자 패턴 (기본: *.jpg)
    label_ext : str
        라벨 확장자 패턴 (기본: *.txt)
    ignore_labels : tuple
        무시할 라벨 파일 이름들
    verbose : bool
        True면 폴더별 상세 출력
    """

    jpg_only_total = []
    txt_only_total = []

    subdirs = sorted([p for p in base_dir.iterdir() if p.is_dir()])

    for subdir in subdirs:
        jpg_stems = {f.stem for f in subdir.glob(image_ext)}
        txt_stems = {f.stem for f in subdir.glob(label_ext)
                     if f.name not in ignore_labels}

        jpg_only = sorted(jpg_stems - txt_stems)
        txt_only = sorted(txt_stems - jpg_stems)

        if verbose and (jpg_only or txt_only):
            print(f"\n📂 폴더: {subdir.name}")

            if jpg_only:
                print("  📸 JPG만 있고 대응되는 TXT 없는 파일:")
                for stem in jpg_only:
                    print(f"    {stem}.jpg")

            if txt_only:
                print("  📝 TXT만 있고 대응되는 JPG 없는 파일:")
                for stem in txt_only:
                    print(f"    {stem}.txt")

        # 절대 경로 수집
        jpg_only_total.extend(subdir / f"{stem}.jpg" for stem in jpg_only)
        txt_only_total.extend(subdir / f"{stem}.txt" for stem in txt_only)

    # 요약 출력
    print("\n==============================")
    print(f"📸 JPG만 있는 파일: {len(jpg_only_total)}개")
    print(f"📝 TXT만 있는 파일: {len(txt_only_total)}개")
    print("==============================")

    return jpg_only_total, txt_only_total


# ============================
# 사용 예시
# ============================

BASE_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")

jpg_only, txt_only = check_label_matching(BASE_DIR)

### 2-1. clustering_LBP + ZNCC

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
순차 스캔: 조명 영향 제거(밝기/대비 불변) 기준 + 드리프트 억제(앵커 비교) + 폴더별 최대 길이 제한
- 파일명 자연정렬
- LBP 히스토그램 + ZNCC 사용 (밝기/대비 변화에 둔감)
- change = GAMMA * d_lbp + (1-GAMMA) * (1-zncc)
- (prev 또는 anchor 기준 change) >= CHANGE_THRESH 이면 새 그룹
- 폴더별 파라미터 오버라이드(FOLDER_CFG) + 폴더별 MAX_CLUSTER_LEN
- MAX_CLUSTER_LEN 도달 시 유사도와 무관하게 새 클러스터 시작(앵커 갱신)

CSV: clusters_invariant_<basename>.csv
  filename, cluster_id, index_in_folder,
  d_lbp_prev, zncc_prev, change_prev,
  d_lbp_anchor, zncc_anchor, change_anchor,
  is_cluster_start, cluster_len_so_far, reason
"""

from pathlib import Path
import cv2
import numpy as np
import csv, re
from typing import Optional, List, Dict, Tuple

# ================================================================
# 🧩 기본 설정
# ================================================================
BASE_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")

SUBFOLDERS = [
    "20250721_good_data",
    "20250725_good_data",
    "20250904_good_data",
    "20250929_good_data",
    "20250930_good_data",
]

# 자동 결합
FOLDERS = [BASE_DIR / name for name in SUBFOLDERS if (BASE_DIR / name).exists()]
missing = [name for name in SUBFOLDERS if not (BASE_DIR / name).exists()]
print(f"[OK] 대상 폴더 {len(FOLDERS)}개:")
for f in FOLDERS:
    print("  └", f)
if missing:
    print(f"[WARN] 존재하지 않는 폴더: {missing}")

# ================================================================
# 폴더별 파라미터
# ================================================================
FOLDER_CFG: Dict[str, Dict] = {
    "20250721_good_data": {"GAMMA": 0.60, "CHANGE_THRESH": 0.32, "GAUSS_BLUR": (3,3), "ZNCC_SIZE": 256, "MAX_CLUSTER_LEN": 5},
    "20250725_good_data": {"GAMMA": 0.60, "CHANGE_THRESH": 0.35, "GAUSS_BLUR": (3,3), "ZNCC_SIZE": 256, "MAX_CLUSTER_LEN": 5},
    "20250904_good_data": {"GAMMA": 0.55, "CHANGE_THRESH": 0.25, "GAUSS_BLUR": (3,3), "ZNCC_SIZE": 256, "MAX_CLUSTER_LEN": 25},
    "20250929_good_data": {"GAMMA": 0.55, "CHANGE_THRESH": 0.25, "GAUSS_BLUR": (3,3), "ZNCC_SIZE": 256, "MAX_CLUSTER_LEN": 7},
    "20250930_good_data": {"GAMMA": 0.60, "CHANGE_THRESH": 0.28, "GAUSS_BLUR": (3,3), "ZNCC_SIZE": 256, "MAX_CLUSTER_LEN": 15},
}

DEFAULT_CFG = {"GAMMA": 0.60, "CHANGE_THRESH": 0.20, "GAUSS_BLUR": (3,3), "ZNCC_SIZE": 256, "MAX_CLUSTER_LEN": 5}

# ================================================================
# 이미지/기타 설정
# ================================================================
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# ====== LBP 설정 ======
LBP_RADIUS = 1
LBP_POINTS = 8
LBP_HIST_BINS = 256  # 8-bit LBP

# ====== 기타 ======
NATURAL_SORT = True
MAX_IMAGES = None
DOWNSCALE_IF_NEEDED = False
PREPROC_MAX_SIDE = 0   # 0=축소 안함(정확도 우선)

# ================================================================
# 유틸 함수
# ================================================================
def get_cfg(folder: Path) -> Dict:
    return {**DEFAULT_CFG, **FOLDER_CFG.get(folder.name, {})}

def natural_key(s: str):
    if not NATURAL_SORT:
        return s
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", s)]

def list_images(folder: Path) -> List[Path]:
    files = [p for p in folder.iterdir() if p.is_file() and p.suffix in IMG_EXTS]
    files.sort(key=lambda p: natural_key(p.name))
    return files[:MAX_IMAGES] if MAX_IMAGES else files

def resize_longest(img: np.ndarray, max_side: int) -> np.ndarray:
    if max_side <= 0:
        return img
    h, w = img.shape[:2]
    longest = max(h, w)
    if longest <= max_side:
        return img
    scale = max_side / float(longest)
    return cv2.resize(img, (int(round(w * scale)), int(round(h * scale))), interpolation=cv2.INTER_AREA)

def load_bgr(path: Path) -> Optional[np.ndarray]:
    img = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img is None:
        return None
    if DOWNSCALE_IF_NEEDED and PREPROC_MAX_SIDE > 0:
        img = resize_longest(img, PREPROC_MAX_SIDE)
    return img

# ================================================================
# LBP / ZNCC 계산
# ================================================================
def lbp_image(gray: np.ndarray) -> np.ndarray:
    """간단 3x3 LBP (8-bit)."""
    g = gray
    c = g[1:-1, 1:-1]
    code = np.zeros_like(c, dtype=np.uint8)
    nbrs = [
        (0, 0),  (0, 1),  (0, 2),
        (1, 2),  (2, 2),  (2, 1),
        (2, 0),  (1, 0),
    ]
    for bit, (dy, dx) in enumerate(nbrs):
        n = g[dy:dy + c.shape[0], dx:dx + c.shape[1]]
        code |= ((n >= c) << (7 - bit)).astype(np.uint8)
    return code

def lbp_hist(gray: np.ndarray, blur_kernel: Tuple[int,int]) -> np.ndarray:
    """정규화된 LBP 히스토그램(밝기/대비 변화에 둔감)."""
    if blur_kernel and blur_kernel[0] > 0 and blur_kernel[1] > 0:
        gray = cv2.GaussianBlur(gray, blur_kernel, 0)
    lbp = lbp_image(gray)
    hist = cv2.calcHist([lbp], [0], None, [LBP_HIST_BINS], [0, 256])
    cv2.normalize(hist, hist)
    return hist

def d_bhat(h1: np.ndarray, h2: np.ndarray) -> float:
    return float(cv2.compareHist(h1, h2, cv2.HISTCMP_BHATTACHARYYA))

def zncc(gray1: np.ndarray, gray2: np.ndarray, size: int, blur_kernel: Tuple[int,int]) -> float:
    """Zero-mean NCC (밝기/대비 선형 변화 불변)."""
    g1 = cv2.resize(gray1, (size, size), interpolation=cv2.INTER_AREA)
    g2 = cv2.resize(gray2, (size, size), interpolation=cv2.INTER_AREA)
    if blur_kernel and blur_kernel[0] > 0 and blur_kernel[1] > 0:
        g1 = cv2.GaussianBlur(g1, blur_kernel, 0)
        g2 = cv2.GaussianBlur(g2, blur_kernel, 0)
    g1, g2 = g1.astype(np.float32), g2.astype(np.float32)
    g1 -= g1.mean(); g2 -= g2.mean()
    std1 = g1.std() + 1e-6
    std2 = g2.std() + 1e-6
    g1 /= std1; g2 /= std2
    num = float(np.sum(g1 * g2))
    den = float(np.sqrt(np.sum(g1*g1)) * np.sqrt(np.sum(g2*g2)) + 1e-6)
    return float(np.clip(num / den, -1.0, 1.0))

def gray_of(img_bgr: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

# ================================================================
# 클러스터링 메인 함수
# ================================================================
def cluster_folder(folder: Path):
    cfg = get_cfg(folder)
    GAMMA = float(cfg["GAMMA"])
    CHANGE_THRESH = float(cfg["CHANGE_THRESH"])
    GAUSS_BLUR = tuple(cfg["GAUSS_BLUR"])
    ZNCC_SIZE = int(cfg["ZNCC_SIZE"])
    MAX_CLUSTER_LEN = int(cfg["MAX_CLUSTER_LEN"])

    imgs = list_images(folder)
    if not imgs:
        print(f"[WARN] No images in: {folder}")
        return

    img_prev = load_bgr(imgs[0])
    if img_prev is None:
        print(f"[WARN] Can't read: {imgs[0]}")
        return

    g_prev = gray_of(img_prev)
    lbp_prev = lbp_hist(g_prev, GAUSS_BLUR)

    anchor_img, g_anchor, lbp_anchor = img_prev, g_prev, lbp_prev
    cluster_id, curr_len = 0, 1

    rows = [{
        "filename": imgs[0].name, "cluster_id": 0, "index_in_folder": 0,
        "d_lbp_prev": 0.0, "zncc_prev": 1.0, "change_prev": 0.0,
        "d_lbp_anchor": 0.0, "zncc_anchor": 1.0, "change_anchor": 0.0,
        "is_cluster_start": 1, "cluster_len_so_far": 1, "reason": "start"
    }]

    for idx in range(1, len(imgs)):
        path = imgs[idx]
        img = load_bgr(path)
        if img is None:
            print(f"[WARN] Can't read: {path}")
            continue

        g_cur = gray_of(img)
        lbp_cur = lbp_hist(g_cur, GAUSS_BLUR)

        d_prev = d_bhat(lbp_prev, lbp_cur)
        z_prev = zncc(g_prev, g_cur, ZNCC_SIZE, GAUSS_BLUR)
        change_prev = GAMMA * d_prev + (1 - GAMMA) * (1 - z_prev)

        d_anchor = d_bhat(lbp_anchor, lbp_cur)
        z_anchor = zncc(g_anchor, g_cur, ZNCC_SIZE, GAUSS_BLUR)
        change_anchor = GAMMA * d_anchor + (1 - GAMMA) * (1 - z_anchor)

        limit_hit = (MAX_CLUSTER_LEN > 0) and (curr_len >= MAX_CLUSTER_LEN)
        if limit_hit or (change_prev >= CHANGE_THRESH) or (change_anchor >= CHANGE_THRESH):
            cluster_id += 1
            is_start, reason, curr_len = 1, ("limit" if limit_hit else "new"), 1
            anchor_img, g_anchor, lbp_anchor = img, g_cur, lbp_cur
        else:
            is_start, reason, curr_len = 0, "similar", curr_len + 1

        rows.append({
            "filename": path.name, "cluster_id": cluster_id, "index_in_folder": idx,
            "d_lbp_prev": round(d_prev, 6), "zncc_prev": round(z_prev, 6), "change_prev": round(change_prev, 6),
            "d_lbp_anchor": round(d_anchor, 6), "zncc_anchor": round(z_anchor, 6), "change_anchor": round(change_anchor, 6),
            "is_cluster_start": is_start, "cluster_len_so_far": curr_len, "reason": reason
        })

        img_prev, g_prev, lbp_prev = img, g_cur, lbp_cur

    out_csv = folder.parent / f"clusters_invariant_{folder.name}.csv"
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
        w.writeheader()
        w.writerows(rows)

    n_clusters = (rows[-1]["cluster_id"] if rows else -1) + 1
    print(f"[OK] {folder.name}: {len(rows)} images → {n_clusters} clusters "
          f"(GAMMA={GAMMA}, TH={CHANGE_THRESH}, MAXLEN={MAX_CLUSTER_LEN}) → {out_csv}")

# ================================================================
# 실행
# ================================================================
def main():
    for folder in FOLDERS:
        if not folder.exists():
            print(f"[WARN] Not found: {folder}")
            continue
        cluster_folder(folder)

if __name__ == "__main__":
    main()


### 3. CSV로 파일이름 변경하기

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import csv
import re

# ===== 설정 =====
CSV_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")
CSV_GLOB = "clusters_invariant_*.csv"
IMG_ROOT = CSV_DIR

APPLY_CHANGES = True # 실제 적용하려면 True

# suffix: "_cluster_<id>"
def make_suffix(cluster_id: int) -> str:
    return f"_cluster_{cluster_id}"

# 이미 붙어있는지 체크 (basename 끝이 _cluster_숫자)
SUFFIX_RE = re.compile(r"_cluster_(\d+)$", re.IGNORECASE)

def with_seq(p: Path, k: int) -> Path:
    return p.with_name(f"{p.stem}__{k}{p.suffix}")

def parse_folder_from_csv(csv_path: Path) -> Path:
    m = re.match(r"clusters_invariant_(.+)\.csv$", csv_path.name)
    if not m:
        raise ValueError(f"CSV 이름 형식이 아님: {csv_path.name}")
    folder_name = m.group(1)
    return IMG_ROOT / folder_name

def ensure_nonconflicting(target: Path) -> Path:
    if not target.exists():
        return target
    k = 2
    cand = with_seq(target, k)
    while cand.exists():
        k += 1
        cand = with_seq(target, k)
    return cand

def plan_for_csv(csv_path: Path):
    folder = parse_folder_from_csv(csv_path)
    if not folder.exists():
        print(f"[WARN] 폴더 없음: {folder}  (CSV={csv_path.name})")
        return []

    plans = []  # list of dicts: {old_jpg, new_jpg, old_txt, new_txt}
    with csv_path.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        if "filename" not in reader.fieldnames or "cluster_id" not in reader.fieldnames:
            raise ValueError(f"[ERR] CSV에 filename/cluster_id 없음: {csv_path}")

        for row in reader:
            fn = row["filename"]
            cid = int(row["cluster_id"])

            # jpg만 처리
            if not fn.lower().endswith(".jpg"):
                continue

            jpg_path = folder / fn
            if not jpg_path.exists():
                continue

            stem = jpg_path.stem
            # 이미 _cluster_xxx 붙어있으면 스킵
            if SUFFIX_RE.search(stem):
                continue

            txt_path = folder / f"{stem}.txt"
            suffix = make_suffix(cid)
            new_stem = f"{stem}{suffix}"

            new_jpg = jpg_path.with_name(f"{new_stem}{jpg_path.suffix}")
            new_txt = (txt_path.with_name(f"{new_stem}{txt_path.suffix}")
                       if txt_path.exists() else None)

            plans.append({
                "old_jpg": jpg_path,
                "new_jpg": new_jpg,
                "old_txt": txt_path if txt_path.exists() else None,
                "new_txt": new_txt
            })
    return plans

def run():
    csvs = sorted(CSV_DIR.glob(CSV_GLOB))
    if not csvs:
        print(f"[WARN] CSV 없음: {CSV_DIR}/{CSV_GLOB}")
        return

    total_sets = 0
    for csv_path in csvs:
        plans = plan_for_csv(csv_path)
        if not plans:
            print(f"[INFO] 변경 계획 없음: {csv_path.name}")
            continue

        folder = parse_folder_from_csv(csv_path)
        print(f"\n[PLAN] {csv_path.name} → 폴더: {folder}")
        for p in plans:
            old_jpg, new_jpg = p["old_jpg"], p["new_jpg"]
            old_txt, new_txt = p["old_txt"], p["new_txt"]

            safe_new_jpg = ensure_nonconflicting(new_jpg)
            safe_new_txt = ensure_nonconflicting(new_txt) if new_txt else None

            # 출력(계획)
            if old_txt and safe_new_txt:
                print(f"  - JPG: {old_jpg.name} → {safe_new_jpg.name}")
                print(f"    TXT: {old_txt.name} → {safe_new_txt.name}")
            else:
                print(f"  - JPG: {old_jpg.name} → {safe_new_jpg.name} (txt 없음, jpg만 변경)")

            if APPLY_CHANGES:
                # txt가 있으면 반드시 같이 변경
                if old_txt and safe_new_txt:
                    old_txt.rename(safe_new_txt)
                    # txt 변경 후 jpg 타겟 충돌 재확인
                    safe_new_jpg = ensure_nonconflicting(safe_new_jpg)
                    old_jpg.rename(safe_new_jpg)
                else:
                    # txt가 없으면 jpg만 변경
                    old_jpg.rename(safe_new_jpg)
                total_sets += 1

    if APPLY_CHANGES:
        print(f"\n[OK] 실제 변경 완료. 총 {total_sets}건 처리.")
    else:
        print("\n[DRY-RUN] 실제 변경 없음. APPLY_CHANGES=True 로 적용하세요.")

if __name__ == "__main__":
    run()

### 4-1. 데이터분할하기1 class( 0 1 2 ) 위주로 cluster별로 분할

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import re
import random
import shutil

# ================================================================
# 🧩 기본 설정
# ================================================================
BASE_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251024_merge_data")
SUBFOLDERS = [
    "20250721_good_data",
    "20250725_good_data",
    "20250904_good_data",
    "20250929_good_data",
    "20250930_good_data",
]

# 자동 조합
FOLDERS = [BASE_DIR / name for name in SUBFOLDERS if (BASE_DIR / name).exists()]
missing = [name for name in SUBFOLDERS if not (BASE_DIR / name).exists()]
print(f"[OK] 대상 폴더 {len(FOLDERS)}개:")
for f in FOLDERS:
    print("  └", f)
if missing:
    print(f"[WARN] 존재하지 않는 폴더: {missing}")

# ================================================================
# ⚙️ 파라미터
# ================================================================
REPORT_DIR = BASE_DIR
# split 비율 (0=train, 1=val)
RATIOS = {0: 0.86, 1: 0.14}
SPLIT_NAME = {0: "train", 1: "val"}
SEED = 33

BALANCE_CLASS_IDS = [0, 1, 2]         # 균형 유지용
ALL_CLASS_IDS = list(range(8))        # 리포트 출력용

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
CLUSTER_RE = re.compile(r"^(?P<stem>.+)_cluster_(?P<cid>\d+)(?:__\d+)?$", re.IGNORECASE)

DO_MATERIALIZE = True
MATERIALIZE_MODE = "move"   # "move" | "copy" | "symlink"
CLEAN_PREVIOUS = True
CREATE_EMPTY_LABEL_IF_MISSING = False

IMG_W = 1.0
CLS_W = 1.0
BG_W  = 1.0

# ================================================================
# 🔧 유틸 함수
# ================================================================
def find_images(folder: Path):
    return [p for p in folder.iterdir() if p.is_file() and p.suffix in IMG_EXTS]

def parse_cluster_id(stem: str):
    m = CLUSTER_RE.match(stem)
    return (m.group("stem"), int(m.group("cid"))) if m else (None, None)

def read_label_lines(txt_path: Path):
    if not txt_path.exists():
        return []
    try:
        with txt_path.open("r", encoding="utf-8") as f:
            return [ln.strip() for ln in f if ln.strip()]
    except Exception:
        return []

def count_classes_from_txt_lines(lines, class_ids):
    counts = {k: 0 for k in class_ids}
    for s in lines:
        parts = s.split()
        try:
            cid = int(parts[0])
        except Exception:
            continue
        if cid in counts:
            counts[cid] += 1
    return counts

def build_cluster_stats(folder: Path):
    clusters = {}
    for img in find_images(folder):
        _, cid = parse_cluster_id(img.stem)
        if cid is None:
            continue

        txt = img.with_suffix(".txt")
        lines = read_label_lines(txt)
        counts_all = count_classes_from_txt_lines(lines, ALL_CLASS_IDS)
        counts_bal = {k: counts_all.get(k, 0) for k in BALANCE_CLASS_IDS}
        is_bg = (len(lines) == 0)

        if cid not in clusters:
            clusters[cid] = {
                "images": [],
                "class_counts_bal": {k: 0 for k in BALANCE_CLASS_IDS},
                "class_counts_all": {k: 0 for k in ALL_CLASS_IDS},
                "bg_count": 0,
            }
        clusters[cid]["images"].append(img)
        for k in BALANCE_CLASS_IDS:
            clusters[cid]["class_counts_bal"][k] += counts_bal[k]
        for k in ALL_CLASS_IDS:
            clusters[cid]["class_counts_all"][k] += counts_all[k]
        clusters[cid]["bg_count"] += 1 if is_bg else 0
    return clusters

def stratified_split_for_folder(clusters: dict, ratios: dict, balance_ids: list, seed=42):
    random.seed(seed)
    cluster_items = []
    total_imgs, total_cls, total_bg = 0, {k: 0 for k in balance_ids}, 0

    for cid, info in clusters.items():
        sz = len(info["images"])
        total_imgs += sz
        vec = {"imgs": sz, "bg": info.get("bg_count", 0)}
        total_bg += vec["bg"]
        for k in balance_ids:
            vec[k] = info["class_counts_bal"][k]
            total_cls[k] += vec[k]
        cluster_items.append((cid, vec))

    if total_imgs == 0:
        return {}

    targets_imgs = {s: ratios[s] * total_imgs for s in ratios}
    targets_bg   = {s: ratios[s] * total_bg   for s in ratios}
    targets_class = {k: {s: ratios[s] * total_cls[k] for s in ratios} for k in balance_ids}

    cluster_items.sort(key=lambda x: (-x[1]["imgs"], x[0]))
    assigned_imgs = {s: 0.0 for s in ratios}
    assigned_bg   = {s: 0.0 for s in ratios}
    assigned_class = {k: {s: 0.0 for s in ratios} for k in balance_ids}
    mapping = {}

    def score_components(ai_imgs, ai_bg, ai_cls):
        img_resid = sum((ai_imgs[s] - targets_imgs[s])**2 for s in ratios)
        bg_resid  = sum((ai_bg[s]   - targets_bg[s])**2   for s in ratios)
        cls_resid = 0.0
        for k in balance_ids:
            cls_resid += sum((ai_cls[k][s] - targets_class[k][s])**2 for s in ratios)
        return IMG_W * img_resid + BG_W * bg_resid + CLS_W * cls_resid

    def score_after_add(split: int, vec: dict):
        ai_imgs = {s: assigned_imgs[s] + (vec["imgs"] if s == split else 0.0) for s in ratios}
        ai_bg   = {s: assigned_bg[s]   + (vec["bg"]   if s == split else 0.0) for s in ratios}
        ai_cls  = {k: {s: assigned_class[k][s] + (vec[k] if s == split else 0.0) for s in ratios}
                   for k in balance_ids}
        return score_components(ai_imgs, ai_bg, ai_cls)

    for cid, vec in cluster_items:
        best_s, best_sc = None, None
        for s in ratios:
            sc = score_after_add(s, vec)
            if best_sc is None or sc < best_sc:
                best_sc, best_s = sc, s
        mapping[cid] = best_s
        assigned_imgs[best_s] += vec["imgs"]
        assigned_bg[best_s]   += vec["bg"]
        for k in balance_ids:
            assigned_class[k][best_s] += vec[k]
    return mapping

# ================================================================
# 📁 파일 분할 및 리포트
# ================================================================
def ensure_clean_split_dirs(base: Path):
    images_root = base / "images"
    labels_root = base / "labels"
    if CLEAN_PREVIOUS:
        shutil.rmtree(images_root, ignore_errors=True)
        shutil.rmtree(labels_root, ignore_errors=True)
    for split in ("train", "val"):
        (images_root / split).mkdir(parents=True, exist_ok=True)
        (labels_root / split).mkdir(parents=True, exist_ok=True)

def place_file(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if MATERIALIZE_MODE == "move":
        shutil.move(str(src), str(dst))
    elif MATERIALIZE_MODE == "copy":
        shutil.copy2(str(src), str(dst))
    elif MATERIALIZE_MODE == "symlink":
        try:
            dst.symlink_to(src.resolve())
        except FileExistsError:
            dst.unlink()
            dst.symlink_to(src.resolve())

def materialize_and_save_report(folder: Path, clusters: dict, mapping: dict, report_dir: Path):
    ensure_clean_split_dirs(folder)

    split_img_counts = {0:0, 1:0}
    split_bg_counts  = {0:0, 1:0}
    split_class_counts = {s: {k:0 for k in ALL_CLASS_IDS} for s in (0,1)}

    for cid, info in clusters.items():
        sp = mapping[cid]
        img_dir = folder / "images" / SPLIT_NAME[sp]
        lbl_dir = folder / "labels" / SPLIT_NAME[sp]

        for k in ALL_CLASS_IDS:
            split_class_counts[sp][k] += info["class_counts_all"][k]
        split_bg_counts[sp] += info.get("bg_count", 0)

        for img in info["images"]:
            split_img_counts[sp] += 1
            place_file(img, img_dir / img.name)

            txt_src = img.with_suffix(".txt")
            txt_dst = lbl_dir / txt_src.name
            if txt_src.exists():
                place_file(txt_src, txt_dst)
            elif CREATE_EMPTY_LABEL_IF_MISSING:
                txt_dst.touch(exist_ok=True)

    # 리포트 출력
    lines = []
    total = sum(split_img_counts.values())
    lines.append(f"[OK] {folder.name} 분할 완료 (총 {total} images) mode={MATERIALIZE_MODE}")
    for s in (0,1):
        cnt = split_img_counts[s]
        pct = (cnt/total)*100 if total else 0
        lines.append(f"  {SPLIT_NAME[s]}: {cnt} imgs ({pct:.1f}%)")

    total_bg = sum(split_bg_counts.values())
    lines.append("\n[BG (no-label) image distribution]")
    for s in (0,1):
        pct = (split_bg_counts[s]/total_bg*100) if total_bg else 0.0
        lines.append(f"  {SPLIT_NAME[s]}: {split_bg_counts[s]} bg ({pct:.1f}%)")

    lines.append("\n[Class distribution (instances per split)]")
    hdr = " ".join([f"{SPLIT_NAME[s]:>10}" for s in (0,1)])
    lines.append(f"{'class':>10} {hdr}")
    for k in ALL_CLASS_IDS:
        row = " ".join([f"{split_class_counts[s][k]:>10d}" for s in (0,1)])
        lines.append(f"{k:>10} {row}")

    report_dir.mkdir(parents=True, exist_ok=True)
    out_txt = report_dir / f"split_report_{folder.name}.txt"
    with out_txt.open("w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")

    print("\n" + "\n".join(lines))
    print(f"\n[Saved] {out_txt}")

# ================================================================
# 🚀 실행
# ================================================================
def run():
    random.seed(SEED)
    REPORT_DIR.mkdir(parents=True, exist_ok=True)

    for folder in FOLDERS:
        if not folder.exists():
            print(f"[WARN] 폴더 없음: {folder}")
            continue

        clusters = build_cluster_stats(folder)
        if not clusters:
            print(f"[INFO] 클러스터 없음(_cluster_<id> 파일 미존재): {folder.name}")
            continue

        mapping = stratified_split_for_folder(clusters, RATIOS, BALANCE_CLASS_IDS, seed=SEED)

        if DO_MATERIALIZE:
            materialize_and_save_report(folder, clusters, mapping, REPORT_DIR)
        else:
            print(f"[DRY] 분할 매핑만 완료(배치 미수행): {folder.name}")

if __name__ == "__main__":
    run()


### 4-2. cluster가 섞였는지 점검

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import re
from collections import defaultdict

# ================================================================
# 🧩 기본 설정
# ================================================================
# merge 날짜만 바꾸면 됨
BASE_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")

# 점검할 하위 폴더 이름만 추가
SUBFOLDERS = [
    "20250721_good_data",
    "20250725_good_data",
    "20250904_good_data",
    "20250929_good_data",
    "20250930_good_data",
]

# 자동 조합
FOLDERS = [BASE_DIR / name for name in SUBFOLDERS if (BASE_DIR / name).exists()]
missing = [name for name in SUBFOLDERS if not (BASE_DIR / name).exists()]
print(f"[OK] 점검 대상 폴더 {len(FOLDERS)}개:")
for f in FOLDERS:
    print("  └", f)
if missing:
    print(f"[WARN] 존재하지 않는 폴더: {missing}")

# ================================================================
# 📸 파일 및 클러스터 점검
# ================================================================
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
CLUSTER_RE = re.compile(r"_cluster_(\d+)", re.IGNORECASE)

def check_folder(folder: Path):
    print(f"\n=== Checking {folder.name} ===")

    cluster_splits = defaultdict(set)

    for split in ("train", "val", "test"):
        img_dir = folder / "images" / split
        if not img_dir.exists():
            continue
        for p in img_dir.rglob("*"):
            if p.is_file() and p.suffix in IMG_EXTS:
                m = CLUSTER_RE.search(p.stem)
                if not m:
                    continue
                cluster_splits[m.group(1)].add(split)

    # 중복 확인
    bad = {cid: sorted(splits) for cid, splits in cluster_splits.items() if len(splits) > 1}
    if not bad:
        print("✅ 모든 클러스터가 한 split에만 존재 (문제 없음)")
    else:
        print(f"❌ {len(bad)}개의 클러스터가 여러 split에 섞여 있음")
        for cid, splits in bad.items():
            print(f"  cluster {cid}: {', '.join(splits)}")

# ================================================================
# 🚀 실행
# ================================================================
if __name__ == "__main__":
    for f in FOLDERS:
        check_folder(f)


### 5.데이터분할확인_1

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
from collections import Counter, defaultdict

# ================================================================
# 🧩 기본 설정
# ================================================================
ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")
TARGETS = [
    ROOT / "20250721_good_data",
    ROOT / "20250725_good_data",
    ROOT / "20250904_good_data",
    ROOT / "20250929_good_data",
    ROOT / "20250930_good_data",
]
SPLITS = ["train", "val"]  # ✅ test 제거
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# 보고서 파일
out_file = ROOT / "split_class_report_012.txt"

# ================================================================
# ⚙️ 유틸 함수
# ================================================================
def read_classes_txt(folder: Path):
    """classes.txt가 있으면 그 순서를 그대로 사용, 없으면 0~7"""
    f = folder / "classes.txt"
    if f.exists():
        names = [ln.strip() for ln in f.read_text(encoding="utf-8").splitlines() if ln.strip()]
        return {i: names[i] for i in range(len(names))}
    return {i: f"class_{i}" for i in range(8)}

def list_images(dirpath: Path):
    """이미지 파일 목록"""
    return [p for p in dirpath.rglob("*") if p.suffix in IMG_EXTS]

def count_split(folder: Path, split: str, id2name: dict):
    """한 데이터셋의 train/val split에 대해 통계 계산"""
    img_dir, lbl_dir = folder / "images" / split, folder / "labels" / split
    cls_counter = Counter()
    total_labels = 0

    # ---- 클래스 카운트 ----
    if lbl_dir.exists():
        for txt in lbl_dir.rglob("*.txt"):
            for line in txt.read_text(encoding="utf-8", errors="ignore").splitlines():
                if not line.strip():
                    continue
                parts = line.split()
                try:
                    cid = int(float(parts[0]))
                except:
                    continue
                cls_counter[cid] += 1
                total_labels += 1

    # ---- BG 및 이미지 수 ----
    bg = 0
    stems_lbl = set()
    if img_dir.exists():
        imgs = list_images(img_dir)
        stems_img = {p.stem for p in imgs}
        stems_lbl = {p.stem for p in lbl_dir.rglob("*.txt")} if lbl_dir.exists() else set()
        bg = len(stems_img - stems_lbl)

    total_images = len(list_images(img_dir)) if img_dir.exists() else 0
    labeled_images = len(stems_lbl) if lbl_dir.exists() else 0
    per_class = {id2name.get(k, f"class_{k}"): v for k, v in sorted(cls_counter.items())}
    return per_class, bg, total_images, labeled_images, total_labels

# ================================================================
# 📊 전체 집계용
# ================================================================
global_total_images = 0
global_total_labeled = 0
global_total_bg = 0
global_total_objects = 0
global_class_totals = Counter()
all_class_names_seen = set()
report_lines = []

PREFERRED_ORDER = [
    "Divot", "Fixed_Divot", "Diseased_Grass", "Confused_Object",
    "Pole", "Sprinkler", "Drain", "Golf ball"
]

# ================================================================
# 📁 데이터셋별 루프
# ================================================================
for tgt in TARGETS:
    id2name = read_classes_txt(tgt)
    all_class_names_seen.update(id2name.values())

    report_lines.append(f"\n=== {tgt.name} ===")
    for split in SPLITS:
        per_class, bg, total_images, labeled_images, total_labels = count_split(tgt, split, id2name)

        # 상세 라인
        report_lines.append(
            f"[{split}] images={total_images}, labeled={labeled_images}, BG(no-label)={bg}, total_objects={total_labels}"
        )
        for i in sorted(id2name.keys()):
            cname = id2name[i]
            report_lines.append(f"  {cname:16s}: {per_class.get(cname, 0)}")
        report_lines.append(f"  {'BG':16s}: {bg}")

        # 총합 집계
        global_total_images += total_images
        global_total_labeled += labeled_images
        global_total_bg += bg
        global_total_objects += total_labels
        for cname, cnt in per_class.items():
            global_class_totals[cname] += cnt

# ================================================================
# 🧾 TOTAL 섹션 작성
# ================================================================
class_order = [c for c in PREFERRED_ORDER if c in all_class_names_seen] + \
              [c for c in sorted(all_class_names_seen) if c not in PREFERRED_ORDER]

total_lines = []
total_lines.append("=== TOTAL (ALL DATASETS, train/val only) ===")
total_lines.append(
    f"images={global_total_images}, labeled={global_total_labeled}, "
    f"BG(no-label)={global_total_bg}, total_objects={global_total_objects}"
)
for cname in class_order:
    total_lines.append(f"  {cname:16s}: {global_class_totals.get(cname, 0)}")
total_lines.append(f"  {'BG':16s}: {global_total_bg}")

# ================================================================
# 💾 파일 저장 + 콘솔 요약
# ================================================================
final_text = "\n".join(total_lines + [""] + report_lines)
out_file.write_text(final_text, encoding="utf-8")

print(f"✅ 전체 결과를 {out_file} 에 저장했습니다.\n")
print("[TOP SUMMARY]")
for line in total_lines:
    print(line)

print("\n[Per-dataset quick image counts]")
for tgt in TARGETS:
    for split in SPLITS:
        img_count = len(list_images(tgt / "images" / split))
        print(f"{tgt.name} [{split}] images: {img_count}")


### 6-1. cluster분할 class ( 4 5 6 7 )

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
import random, shutil

# ===== 기본 경로 설정 =====
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")
FOLDERS = [
    BASE / "20250721_good_data",
    BASE / "20250725_good_data",
    BASE / "20250904_good_data",
    BASE / "20250929_good_data",
    BASE / "20250930_good_data",
]
REPORT_DIR = BASE

# ===== 분할 설정 =====
RATIOS = {0: 0.86, 1: 0.14}     # train:val 비율 (클래스별)
SPLIT_NAME = {0: "train", 1: "val"}
SEED = 33

# ===== 대상 클래스 / 조건 =====
TARGET_CLASS_IDS = [4, 5, 6, 7]
EXCLUDE_IF_HAS_012 = False
HANDLE_BG = True

# ===== 실제 반영 방식 =====
MATERIALIZE_MODE = "move"       # move | copy | symlink
DRY_RUN = False                 # True면 이동 미리보기만

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# =========================================================
def split_dirs(base: Path):
    return {
        0: (base / "images" / "train", base / "labels" / "train"),
        1: (base / "images" / "val",   base / "labels" / "val"),
    }

def iter_split_images(base: Path):
    for sid, (dimg, _) in split_dirs(base).items():
        if not dimg.exists():
            continue
        for p in dimg.iterdir():
            if p.is_file() and p.suffix in IMG_EXTS:
                yield sid, p

def read_label_lines(txt: Path):
    if not txt.exists():
        return []
    try:
        return [ln.strip() for ln in txt.read_text(encoding="utf-8").splitlines() if ln.strip()]
    except:
        return []

def get_class_counts_and_flags(txt: Path):
    lines = read_label_lines(txt)
    is_bg = (len(lines) == 0)
    counts = {k: 0 for k in TARGET_CLASS_IDS}
    has_012 = False
    for s in lines:
        parts = s.split()
        try:
            cid = int(parts[0])
        except:
            continue
        if cid in counts:
            counts[cid] += 1
        if cid in (0, 1, 2):
            has_012 = True
    return counts, is_bg, has_012

def place_file(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if MATERIALIZE_MODE == "move":
        if src.resolve() == dst.resolve():
            return
        shutil.move(str(src), str(dst))
    elif MATERIALIZE_MODE == "copy":
        shutil.copy2(str(src), str(dst))
    elif MATERIALIZE_MODE == "symlink":
        try:
            dst.symlink_to(src.resolve())
        except FileExistsError:
            dst.unlink(missing_ok=True)
            dst.symlink_to(src.resolve())
    else:
        raise ValueError("MATERIALIZE_MODE must be move/copy/symlink")

# =========================================================
def plan_and_apply(folder: Path):
    random.seed(SEED)

    # 1) 대상 이미지 수집
    items = []   # (img, lbl, from_split, vec)
    totals = {"imgs": 0, "bg": 0, **{k: 0 for k in TARGET_CLASS_IDS}}

    for sid, img in iter_split_images(folder):
        lbl = folder / "labels" / SPLIT_NAME[sid] / (img.stem + ".txt")
        cnts, is_bg, has_012 = get_class_counts_and_flags(lbl)

        if EXCLUDE_IF_HAS_012 and has_012:
            continue
        has_target = sum(cnts.values()) > 0
        if not has_target and not (HANDLE_BG and is_bg):
            continue

        vec = {"imgs": 1, "bg": 1 if (HANDLE_BG and is_bg) else 0}
        vec.update(cnts)
        items.append((img, lbl, sid, vec))

        totals["imgs"] += 1
        totals["bg"] += vec["bg"]
        for k in TARGET_CLASS_IDS:
            totals[k] += cnts[k]

    if not items:
        print(f"[INFO] {folder.name}: 대상(4/5/6/7 + BG) 없음")
        return

    # 2) 클래스별 총 객체 수
    class_totals = {k: sum(it[3][k] for it in items) for k in TARGET_CLASS_IDS}
    for k, v in class_totals.items():
        if v == 0:
            print(f"[WARN] {folder.name}: 클래스 {k} 객체가 없음")
    target_objs = {s: {k: class_totals[k] * RATIOS[s] for k in TARGET_CLASS_IDS} for s in RATIOS}

    # 3) 클래스별 비율 기반 배정
    random.shuffle(items)
    obj_counts = {s: {k: 0 for k in TARGET_CLASS_IDS} for s in RATIOS}
    assignment = []

    for it in items:
        # 각 split의 부족도 계산 (모든 클래스 평균)
        score = {}
        for s in RATIOS:
            diff_sum = 0
            for k in TARGET_CLASS_IDS:
                # 목표 대비 현재 차이 (양수면 초과, 음수면 부족)
                diff = obj_counts[s][k] - target_objs[s][k]
                # 객체가 많은 클래스일수록 더 민감하게
                diff_sum += diff / max(class_totals[k], 1)
            score[s] = diff_sum
        # diff_sum이 더 작은 (즉, 더 부족한) split으로 보냄
        best = min(score, key=score.get)

        assignment.append((it[0], it[1], it[2], best, it[3]))
        for k in TARGET_CLASS_IDS:
            obj_counts[best][k] += it[3][k]

    # 4) 콘솔 요약
    print(f"\n=== {'DRY-RUN' if DRY_RUN else 'APPLY'}: 클래스별(4/5/6/7) 9:1 재분할 → {folder.name} ===")
    move_preview = sum(1 for img, lbl, from_s, to_s, _ in assignment if from_s != to_s)
    print(f"대상 이미지: {len(items)} / 이동 예정: {move_preview}")

    for k in TARGET_CLASS_IDS:
        total = class_totals[k]
        t_train, t_val = obj_counts[0][k], obj_counts[1][k]
        pct_train = (t_train / total * 100) if total else 0
        pct_val = (t_val / total * 100) if total else 0
        print(f"Class {k}: total={total}, train={t_train} ({pct_train:.1f}%), val={t_val} ({pct_val:.1f}%)")

    # 5) 실제 반영
    if not DRY_RUN:
        for img, lbl, from_s, to_s, vec in assignment:
            if from_s == to_s:
                continue
            img_dst = folder / "images" / SPLIT_NAME[to_s] / img.name
            lbl_dst = folder / "labels" / SPLIT_NAME[to_s] / (img.stem + ".txt")
            place_file(img, img_dst)
            if lbl.exists():
                place_file(lbl, lbl_dst)

    # 6) 결과 리포트
    lines = []
    lines.append(f"[{'DRY' if DRY_RUN else 'OK'}] ({'plan only' if DRY_RUN else 'applied'}) "
                 f"{folder.name} – 클래스별 객체 수 기준 재분할  mode={MATERIALIZE_MODE}")
    for k in TARGET_CLASS_IDS:
        total = class_totals[k]
        t_train, t_val = obj_counts[0][k], obj_counts[1][k]
        pct_train = (t_train / total * 100) if total else 0
        pct_val = (t_val / total * 100) if total else 0
        lines.append(f"  Class {k}: total={total}, train={t_train} ({pct_train:.1f}%), val={t_val} ({pct_val:.1f}%)")

    REPORT_DIR.mkdir(parents=True, exist_ok=True)
    out_txt = REPORT_DIR / f"{'dry_' if DRY_RUN else ''}applied_objwise_perclass_c4567_{folder.name}.txt"
    with out_txt.open("w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")

    print("\n".join(lines))
    print(f"[Saved] {out_txt}")

# =========================================================
def run():
    for folder in FOLDERS:
        if not folder.exists():
            print(f"[WARN] 폴더 없음: {folder}")
            continue
        sd = split_dirs(folder)
        if not all(sd[s][0].exists() and sd[s][1].exists() for s in (0, 1)):
            print(f"[WARN] YOLO 구조(images/labels/train,val) 없음: {folder}")
            continue
        plan_and_apply(folder)

if __name__ == "__main__":
    run()



### 6-2. 클러스터 점검

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import re
from collections import defaultdict

# ================================================================
# 🧩 기본 설정
# ================================================================
# merge 날짜만 바꾸면 됨
BASE_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")

# 점검할 하위 폴더 이름만 추가
SUBFOLDERS = [
    "20250721_good_data",
    "20250725_good_data",
    "20250904_good_data",
    "20250929_good_data",
    "20250930_good_data",
]

# 자동 조합
FOLDERS = [BASE_DIR / name for name in SUBFOLDERS if (BASE_DIR / name).exists()]
missing = [name for name in SUBFOLDERS if not (BASE_DIR / name).exists()]
print(f"[OK] 점검 대상 폴더 {len(FOLDERS)}개:")
for f in FOLDERS:
    print("  └", f)
if missing:
    print(f"[WARN] 존재하지 않는 폴더: {missing}")

# ================================================================
# 📸 파일 및 클러스터 점검
# ================================================================
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
CLUSTER_RE = re.compile(r"_cluster_(\d+)", re.IGNORECASE)

def check_folder(folder: Path):
    print(f"\n=== Checking {folder.name} ===")

    cluster_splits = defaultdict(set)

    for split in ("train", "val", "test"):
        img_dir = folder / "images" / split
        if not img_dir.exists():
            continue
        for p in img_dir.rglob("*"):
            if p.is_file() and p.suffix in IMG_EXTS:
                m = CLUSTER_RE.search(p.stem)
                if not m:
                    continue
                cluster_splits[m.group(1)].add(split)

    # 중복 확인
    bad = {cid: sorted(splits) for cid, splits in cluster_splits.items() if len(splits) > 1}
    if not bad:
        print("✅ 모든 클러스터가 한 split에만 존재 (문제 없음)")
    else:
        print(f"❌ {len(bad)}개의 클러스터가 여러 split에 섞여 있음")
        for cid, splits in bad.items():
            print(f"  cluster {cid}: {', '.join(splits)}")

# ================================================================
# 🚀 실행
# ================================================================
if __name__ == "__main__":
    for f in FOLDERS:
        check_folder(f)


### 7-1. 데이터 분할 확인_2

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
from collections import Counter, defaultdict

# ================================================================
# 🧩 기본 설정
# ================================================================
ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")
TARGETS = [
    ROOT / "20250721_good_data",
    ROOT / "20250725_good_data",
    ROOT / "20250904_good_data",
    ROOT / "20250929_good_data",
    ROOT / "20250930_good_data",
]
SPLITS = ["train", "val"]  # ✅ test 제거
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# 보고서 파일
out_file = ROOT / "split_class_report_after_4567.txt"

# ================================================================
# ⚙️ 유틸 함수
# ================================================================
def read_classes_txt(folder: Path):
    """classes.txt가 있으면 그 순서를 그대로 사용, 없으면 0~7"""
    f = folder / "classes.txt"
    if f.exists():
        names = [ln.strip() for ln in f.read_text(encoding="utf-8").splitlines() if ln.strip()]
        return {i: names[i] for i in range(len(names))}
    return {i: f"class_{i}" for i in range(8)}

def list_images(dirpath: Path):
    """이미지 파일 목록"""
    return [p for p in dirpath.rglob("*") if p.suffix in IMG_EXTS]

def count_split(folder: Path, split: str, id2name: dict):
    """한 데이터셋의 train/val split에 대해 통계 계산"""
    img_dir, lbl_dir = folder / "images" / split, folder / "labels" / split
    cls_counter = Counter()
    total_labels = 0

    # ---- 클래스 카운트 ----
    if lbl_dir.exists():
        for txt in lbl_dir.rglob("*.txt"):
            for line in txt.read_text(encoding="utf-8", errors="ignore").splitlines():
                if not line.strip():
                    continue
                parts = line.split()
                try:
                    cid = int(float(parts[0]))
                except:
                    continue
                cls_counter[cid] += 1
                total_labels += 1

    # ---- BG 및 이미지 수 ----
    bg = 0
    stems_lbl = set()
    if img_dir.exists():
        imgs = list_images(img_dir)
        stems_img = {p.stem for p in imgs}
        stems_lbl = {p.stem for p in lbl_dir.rglob("*.txt")} if lbl_dir.exists() else set()
        bg = len(stems_img - stems_lbl)

    total_images = len(list_images(img_dir)) if img_dir.exists() else 0
    labeled_images = len(stems_lbl) if lbl_dir.exists() else 0
    per_class = {id2name.get(k, f"class_{k}"): v for k, v in sorted(cls_counter.items())}
    return per_class, bg, total_images, labeled_images, total_labels

# ================================================================
# 📊 전체 집계용
# ================================================================
global_total_images = 0
global_total_labeled = 0
global_total_bg = 0
global_total_objects = 0
global_class_totals = Counter()
all_class_names_seen = set()
report_lines = []

PREFERRED_ORDER = [
    "Divot", "Fixed_Divot", "Diseased_Grass", "Confused_Object",
    "Pole", "Sprinkler", "Drain", "Golf ball"
]

# ================================================================
# 📁 데이터셋별 루프
# ================================================================
for tgt in TARGETS:
    id2name = read_classes_txt(tgt)
    all_class_names_seen.update(id2name.values())

    report_lines.append(f"\n=== {tgt.name} ===")
    for split in SPLITS:
        per_class, bg, total_images, labeled_images, total_labels = count_split(tgt, split, id2name)

        # 상세 라인
        report_lines.append(
            f"[{split}] images={total_images}, labeled={labeled_images}, BG(no-label)={bg}, total_objects={total_labels}"
        )
        for i in sorted(id2name.keys()):
            cname = id2name[i]
            report_lines.append(f"  {cname:16s}: {per_class.get(cname, 0)}")
        report_lines.append(f"  {'BG':16s}: {bg}")

        # 총합 집계
        global_total_images += total_images
        global_total_labeled += labeled_images
        global_total_bg += bg
        global_total_objects += total_labels
        for cname, cnt in per_class.items():
            global_class_totals[cname] += cnt

# ================================================================
# 🧾 TOTAL 섹션 작성
# ================================================================
class_order = [c for c in PREFERRED_ORDER if c in all_class_names_seen] + \
              [c for c in sorted(all_class_names_seen) if c not in PREFERRED_ORDER]

total_lines = []
total_lines.append("=== TOTAL (ALL DATASETS, train/val only) ===")
total_lines.append(
    f"images={global_total_images}, labeled={global_total_labeled}, "
    f"BG(no-label)={global_total_bg}, total_objects={global_total_objects}"
)
for cname in class_order:
    total_lines.append(f"  {cname:16s}: {global_class_totals.get(cname, 0)}")
total_lines.append(f"  {'BG':16s}: {global_total_bg}")

# ================================================================
# 💾 파일 저장 + 콘솔 요약
# ================================================================
final_text = "\n".join(total_lines + [""] + report_lines)
out_file.write_text(final_text, encoding="utf-8")

print(f"✅ 전체 결과를 {out_file} 에 저장했습니다.\n")
print("[TOP SUMMARY]")
for line in total_lines:
    print(line)

print("\n[Per-dataset quick image counts]")
for tgt in TARGETS:
    for split in SPLITS:
        img_count = len(list_images(tgt / "images" / split))
        print(f"{tgt.name} [{split}] images: {img_count}")


### 7-2. cluster 장수

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import re
from collections import defaultdict, Counter

# ================================================================
# 🧩 기본 설정
# ================================================================
BASE_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")

SUBFOLDERS = [
    "20250721_good_data",
    "20250725_good_data",
    "20250904_good_data",
    "20250929_good_data",
    "20250930_good_data",
]

FOLDERS = [BASE_DIR / name for name in SUBFOLDERS if (BASE_DIR / name).exists()]
missing = [name for name in SUBFOLDERS if not (BASE_DIR / name).exists()]

print(f"[OK] 점검 대상 폴더 {len(FOLDERS)}개:")
for f in FOLDERS:
    print("  └", f)
if missing:
    print(f"[WARN] 존재하지 않는 폴더: {missing}")

# ================================================================
# 📸 파일 및 클러스터 점검
# ================================================================
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
CLUSTER_RE = re.compile(r"_cluster_(\d+)", re.IGNORECASE)

def check_folder(folder: Path):
    print(f"\n=== Checking {folder.name} ===")

    # cluster_id → split별 카운트
    cluster_split_counts = defaultdict(lambda: Counter())

    for split in ("train", "val", "test"):
        img_dir = folder / "images" / split
        if not img_dir.exists():
            continue
        for p in img_dir.rglob("*"):
            if p.is_file() and p.suffix in IMG_EXTS:
                m = CLUSTER_RE.search(p.stem)
                if not m:
                    continue
                cid = m.group(1)
                cluster_split_counts[cid][split] += 1

    if not cluster_split_counts:
        print("⚠️  클러스터 패턴(_cluster_###)이 포함된 이미지가 없음")
        return

    # split 혼재된 클러스터 찾기
    mixed_clusters = {cid: cnts for cid, cnts in cluster_split_counts.items() if len(cnts) > 1}

    # 전체 요약 출력
    print(f"총 {len(cluster_split_counts)}개의 클러스터 발견")
    if mixed_clusters:
        print(f"❌ {len(mixed_clusters)}개의 클러스터가 여러 split에 분포")
    else:
        print("✅ 모든 클러스터가 한 split에만 존재")

    # 상세 출력 (train/val/test 개수)
    print("\ncluster_id | train | val | test | total | status")
    print("-" * 60)
    for cid, cnts in sorted(cluster_split_counts.items(), key=lambda x: int(x[0])):
        train = cnts.get("train", 0)
        val = cnts.get("val", 0)
        test = cnts.get("test", 0)
        total = train + val + test
        status = "⚠️ MIXED" if len(cnts) > 1 else "OK"
        print(f"{cid:>10} | {train:5d} | {val:3d} | {test:4d} | {total:5d} | {status}")

# ================================================================
# 🚀 실행
# ================================================================
if __name__ == "__main__":
    for f in FOLDERS:
        check_folder(f)


### 8-1. 데이터 증강 (version_1)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
from collections import Counter, defaultdict, deque
import random, csv, re
import cv2
import albumentations as A

# 🔥 추가: dup/bg 증강본 필터용 정규식
EXCLUDE_RE = re.compile(r'_(dup|bg)\d+$', re.IGNORECASE)

# ========== 대상 폴더들 ==========
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")

BASES = [
    #BASE / "20250721_good_data",
    BASE / "20250725_good_data",
    #BASE / "20250904_good_data",
    #BASE / "20250929_good_data",
    #BASE / "20250930_good_data",
]

# ========== 공통 설정 ==========
IMG_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"]
SEED = 33
random.seed(SEED)

# 클래스 매핑(8클래스, 필요시 변경)
# 0 Divot, 1 Fixed_Divot, 2 Diseased_Grass, 3 Confused_Object, 4 Pole, 5 Sprinkler, 6 Drain, 7 Golf ball
TARGET = {
    0: 422,  # Divot
    1: 717,  # Fixed_Divot
    2: 38,   # Diseased_Grass
    3: 0,    # Confused_Object
    4: 27,   # Pole
    5: 78,   # Sprinkler
    6: 18,   # Drain
    7: 54,   # Golf ball
}
ALL_CLASS_IDS = list(range(8))
ALLOW_IDS = set(ALL_CLASS_IDS)

# --- 중복 억제 파라미터 ---
MAX_USES_BASE = 1                     # 기본 재사용 상한(낮게)
MAX_USES_BOOST_PER_CLASS = {          # 부족 클래스일 때 가산치
    0: 2, 1: 2, 2: 3, 3: 0, 4: 3, 5: 3, 6: 3, 7: 3
}
MAX_PER_IMAGE_HARD = 1                # 한 원본에서 절대 최대 생성 수
RECENT_COOLDOWN = 5                   # 최근 사용 원본 쿨다운 길이

# 배경(라벨 없는 train 이미지) 증강 배수
BG_AUG_MULTIPLIER = 3                 # 원본 1 + 증강 3 = 총 4배

# ========== 색/감마 변환 정의(플립 제거, 색/감마만) ==========
def _build_color_ops():
    ops = {
        "gamma_bright": A.RandomGamma(gamma_limit=(55, 85), p=1.0),    # 더 밝게
        "gamma_dark":   A.RandomGamma(gamma_limit=(120, 180), p=1.0),  # 더 어둡게
        "warm": A.Compose([
            A.ColorJitter(brightness=0.10, contrast=0.15, saturation=0.08, hue=0.015, p=1.0),
            A.RGBShift(r_shift_limit=(8, 20), g_shift_limit=(-4, 6), b_shift_limit=(-20, -8), p=0.9),
            A.RandomGamma(gamma_limit=(95, 120), p=1.0),
        ], p=1.0),
        "cool": A.Compose([
            A.ColorJitter(brightness=0.08, contrast=0.12, saturation=0.06, hue=0.015, p=1.0),
            A.RGBShift(r_shift_limit=(-20, -8), g_shift_limit=(-6, 6), b_shift_limit=(8, 20), p=0.9),
            A.RandomGamma(gamma_limit=(95, 120), p=1.0),
        ], p=1.0),
        "desat": A.Compose([
            A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=(-45, -25), val_shift_limit=(-5, 10), p=1.0),
            A.RandomGamma(gamma_limit=(90, 115), p=1.0),
        ], p=1.0),
        "satboost": A.Compose([
            A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=(20, 45), val_shift_limit=(-5, 10), p=1.0),
            A.RandomGamma(gamma_limit=(90, 115), p=1.0),
        ], p=1.0),
        "identity": A.Compose([], p=1.0),
    }
    return ops

COLOR_OPS = _build_color_ops()

# 선택 가중치(원본 유지 비활성화: identity 제외)
WEIGHTS = {
    "gamma_bright": 1.0,
    "gamma_dark":   1.0,
    "warm":         1.0,
    "cool":         1.0,
    "desat":        0.8,
    "satboost":     0.8,
    "identity":     0.3,

}

def build_aug(weights: dict) -> A.Compose:
    choices = []
    for name, t in COLOR_OPS.items():
        w = float(weights.get(name, 0.0))
        if w > 0:
            t.p = w
            choices.append(t)
    if not choices:
        choices = [A.NoOp(p=1.0)]

    return A.Compose([
        A.HorizontalFlip(p=0.25),
        A.OneOf(choices, p=1.0),
    ], bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["class_labels"],
        min_visibility=0.25
    ))

aug = build_aug(WEIGHTS)

# ========== 유틸 ==========
def read_yolo_label(lbl: Path):
    boxes, cls = [], []
    if not lbl.exists(): return boxes, cls
    with open(lbl, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 5: continue
            try:
                c = int(float(p[0]))
            except:
                continue
            try:
                x, y, w, h = map(float, p[1:5])
            except:
                continue
            x = min(max(x, 0.0), 1.0); y = min(max(y, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0); h = min(max(h, 0.0), 1.0)
            if w <= 0.0 or h <= 0.0: continue
            boxes.append([x, y, w, h]); cls.append(c)
    return boxes, cls

def write_yolo_label(lbl_path: Path, boxes, cls, *, allow_ids=ALLOW_IDS):
    lbl_path.parent.mkdir(parents=True, exist_ok=True)
    with open(lbl_path, "w") as f:
        for c, (x, y, w, h) in zip(cls, boxes):
            try:
                c_int = int(float(c))
            except:
                continue
            if allow_ids is not None and c_int not in allow_ids:
                continue
            x = float(max(0.0, min(1.0, x)))
            y = float(max(0.0, min(1.0, y)))
            w = float(max(0.0, min(1.0, w)))
            h = float(max(0.0, min(1.0, h)))
            if w <= 0.0 or h <= 0.0:
                continue
            f.write(f"{c_int} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def yolo_sanitize(boxes):
    eps = 1e-6; out = []
    for x, y, w, h in boxes:
        l = max(0.0, min(1.0, x - w/2.0))
        r = max(0.0, min(1.0, x + w/2.0))
        t = max(0.0, min(1.0, y - h/2.0))
        b = max(0.0, min(1.0, y + h/2.0))
        if r - l <= eps or b - t <= eps: continue
        nx = (l + r)/2.0; ny = (t + b)/2.0; nw = (r - l); nh = (b - t)
        nx = min(max(nx, eps), 1.0 - eps); ny = min(max(ny, eps), 1.0 - eps)
        nw = min(max(nw, eps), 1.0 - eps); nh = min(max(nh, eps), 1.0 - eps)
        out.append([nx, ny, nw, nh])
    return out

def collect_background_images(img_dir: Path, lbl_dir: Path):
    stems_lbl = {p.stem for p in lbl_dir.rglob("*.txt")} if lbl_dir.exists() else set()
    bg_images = []
    for ext in IMG_EXTS:
        for img in img_dir.rglob(f"*{ext}"):
            if img.stem not in stems_lbl:
                bg_images.append(img)
    return sorted(bg_images)

def load_image(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise RuntimeError(f"Failed to read image: {path}")
    return img

def next_unique_name(stem: str, used: set, tag: str):
    i = 1
    while True:
        s = f"{stem}_{tag}{i:04d}"
        if s not in used:
            used.add(s)
            return s
        i += 1

def effective_cap_for_file(file_cls_counter: Counter, deficit: Counter):
    cap = MAX_USES_BASE
    best_boost = 0
    for c, n in file_cls_counter.items():
        if n <= 0: continue
        if deficit.get(c, 0) > 0:
            best_boost = max(best_boost, MAX_USES_BOOST_PER_CLASS.get(c, 0))
    return cap + best_boost

def current_counts(label_dir: Path):
    cnt = Counter()
    for lbl in label_dir.rglob("*.txt"):
        _, cls = read_yolo_label(lbl)
        cnt.update(cls)
    return cnt

def list_train_stems(img_dir: Path):
    return {p.stem for p in img_dir.rglob("*") if p.suffix.lower() in IMG_EXTS}

# ========== 메인 루틴 ==========
def run_one_base(BASE: Path):
    IMG_DIR = BASE / "images" / "train"
    LBL_DIR = BASE / "labels" / "train"
    IMG_DIR.mkdir(parents=True, exist_ok=True)
    LBL_DIR.mkdir(parents=True, exist_ok=True)

    # 유니크명 충돌 방지: 이미 존재하는 stem 모두 확보
    used_names = set(list_train_stems(IMG_DIR))

    # 1) 배경 증강
    bg_list = collect_background_images(IMG_DIR, LBL_DIR)
    print(f"\n[{BASE.name}] BG images in train: {len(bg_list)}")

    log_path = BASE / "aug_train_log.csv"
    with open(log_path, "w", newline="") as f:
        csv.writer(f).writerow(["src_img","src_lbl","out_img","out_lbl","add_per_class","totals_after"])

    for bg_img in bg_list:
        try:
            img = load_image(bg_img)
        except Exception:
            continue
        for _ in range(BG_AUG_MULTIPLIER):
            out_stem = next_unique_name(bg_img.stem, used_names, tag="bg")
            out_img = IMG_DIR / f"{out_stem}{bg_img.suffix.lower()}"
            out_lbl = LBL_DIR / f"{out_stem}.txt"
            transformed = aug(image=img, bboxes=[], class_labels=[])
            cv2.imwrite(str(out_img), transformed["image"])
            with open(out_lbl, "w") as f:
                pass
    print("  BG augmentation done.")

    # 2) 라벨 쌍 후보(오직 train)
    label_files = sorted(LBL_DIR.rglob("*.txt"))
    candidates = []  # (img_path, lbl_path, file_counts, stem)
    skipped = 0
    for lbl in label_files:
        boxes, cls_list = read_yolo_label(lbl)
        if not cls_list:
            continue
        img = None
        for ext in IMG_EXTS:
            cand = IMG_DIR / f"{lbl.stem}{ext}"
            if cand.exists():
                img = cand; break
        if img is None:
            skipped += 1
            continue

        # 🔥 추가: dup/bg로 생성된 증강본은 제외 (단, 원래 bg는 허용)
        if EXCLUDE_RE.search(lbl.stem):
            continue

        fcnt = Counter(cls_list)
        candidates.append((img, lbl, fcnt, lbl.stem))
    if not candidates:
        print("  No labeled train pairs. Skip.")
        return
    print(f"  Labeled train pairs: {len(candidates)}, skipped(no image match): {skipped}")

    # 3) 현재(train) 카운트 & 결손
    cur = current_counts(LBL_DIR)
    deficit = Counter({c: max(0, TARGET.get(c, 0) - cur.get(c, 0)) for c in ALL_CLASS_IDS})

    # 4) 반복 증강(중복 억제 포함)
    use_count = defaultdict(int)
    used_once = set()  # ✅ 추가
    all_once_done = False  # ✅ 추가
    recent_imgs = deque(maxlen=RECENT_COOLDOWN)
    MAX_ITERS = 200000
    iters = 0

    def base_score(deficit: Counter, fcnt: Counter):
        return sum(deficit[c] * fcnt.get(c, 0) for c in ALL_CLASS_IDS if deficit[c] > 0)

    while any(deficit[c] > 0 for c in ALL_CLASS_IDS) and iters < MAX_ITERS:
        iters += 1
        best = None; best_score = 0.0

        for img, lbl, fcnt, stem in candidates:
            if not all_once_done and use_count[img] >= 1:
                continue
            if img in recent_imgs:
                continue
            if use_count[img] >= MAX_PER_IMAGE_HARD and not all_once_done:
                continue
            cap = effective_cap_for_file(fcnt, deficit)
            if use_count[img] >= cap and not all_once_done:
                continue
            sc0 = base_score(deficit, fcnt)
            if sc0 <= 0:
                continue
            diversity_w = 1.0 / (1.0 + use_count[img])
            sc = sc0 * diversity_w
            if sc > best_score:
                best_score = sc
                best = (img, lbl, fcnt, stem)

        # ✅ 수정 시작
        if not best:
            # 남은 deficit이 있고, 후보 점수 모두 0이면 재사용 라운드로 전환
            useful_remaining = any(base_score(deficit, fcnt) > 0 for _, _, fcnt, _ in candidates)
            if useful_remaining and not all_once_done:
                print("  No more useful candidates (before all_once_done). Stop.")
                # ✅ 기존 break 대신 → 재사용 라운드로 강제 전환
                print("  🔁 Switching to reuse phase (dup002+).")
                all_once_done = True
                used_once.clear()
                use_count = defaultdict(int)
                recent_imgs.clear()
                continue
            elif not useful_remaining:
                print("  🔁 All originals exhausted, entering reuse round.")
                all_once_done = True
                used_once.clear()
                use_count = defaultdict(int)
                recent_imgs.clear()
                continue
            else:
                print("  🔁 All originals used or exhausted, deficit remains. Resetting for next reuse round.")
                all_once_done = True
                used_once.clear()
                use_count = defaultdict(int)
                recent_imgs.clear()
                continue
        # ✅ 수정 끝

        img_path, lbl_path, fcnt, stem = best
        out_stem = next_unique_name(stem, used_names, tag="dup")
        out_img = IMG_DIR / f"{out_stem}{img_path.suffix.lower()}"
        out_lbl = LBL_DIR / f"{out_stem}.txt"

        img = load_image(img_path)
        boxes0, cls0 = read_yolo_label(lbl_path)
        boxes0 = yolo_sanitize(boxes0)

        transformed = aug(image=img, bboxes=boxes0, class_labels=cls0)
        aug_img = transformed["image"]
        aug_boxes = transformed["bboxes"]
        aug_cls = transformed["class_labels"]

        keep_boxes, keep_cls = [], []
        for (x, y, w, h), c in zip(aug_boxes, aug_cls):
            if w > 0 and h > 0:
                keep_boxes.append([float(x), float(y), float(w), float(h)])
                keep_cls.append(int(float(c)))

        if not keep_cls or sum(Counter(keep_cls)[c] for c in ALL_CLASS_IDS if deficit[c] > 0) <= 0:
            use_count[img_path] += 1
            continue

        cv2.imwrite(str(out_img), aug_img)
        write_yolo_label(out_lbl, keep_boxes, keep_cls, allow_ids=ALLOW_IDS)

        add_cnt = Counter(keep_cls)
        cur.update(add_cnt)
        for c in ALL_CLASS_IDS:
            deficit[c] = max(0, TARGET.get(c, 0) - cur.get(c, 0))

        use_count[img_path] += 1
        used_once.add(img_path)
        if not all_once_done and len(used_once) >= len(candidates):
            all_once_done = True
            print("  ✅ All originals used once, reuse now allowed.")

        recent_imgs.append(img_path)

        add_str = "{" + ", ".join(f"{k}:{add_cnt.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
        tot_str = "{" + ", ".join(f"{k}:{cur.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
        with open(log_path, "a", newline="") as f:
            csv.writer(f).writerow([str(img_path), str(lbl_path), str(out_img), str(out_lbl), add_str, tot_str])

        if iters % 50 == 0 or all(deficit[c]==0 for c in ALL_CLASS_IDS):
            print(f"  [{iters}] cur={dict(cur)} deficit={dict(deficit)} last={out_stem}")

    print("  Final train counts:", dict(cur))
    print("  Remaining deficit:", dict(deficit))
    print(f"  Log: {log_path}")

# ===== 실행 =====
if __name__ == "__main__":
    for base in BASES:
        run_one_base(base)



### 9.데이터 병합하기

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
import shutil, csv

# ================================================================
# 🧩 기본 설정
# ================================================================
MERGE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")
SOURCES = [
    MERGE_ROOT / "20250721_good_data",
    MERGE_ROOT / "20250725_good_data",
    MERGE_ROOT / "20250904_good_data",
    MERGE_ROOT / "20250929_good_data",
    MERGE_ROOT / "20250930_good_data",
]

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# ================================================================
# 📂 디렉토리 준비 (train/val만)
# ================================================================
def ensure_dirs():
    for base in ["images", "labels"]:
        for split in ["train", "val"]:   # ✅ test 제거
            (MERGE_ROOT / base / split).mkdir(parents=True, exist_ok=True)
    (MERGE_ROOT / "logs").mkdir(parents=True, exist_ok=True)
    (MERGE_ROOT / "logs" / "classes_sources").mkdir(parents=True, exist_ok=True)

# ================================================================
# 🔧 파일 이동 관련
# ================================================================
def map_images(dir_path: Path):
    return {p.stem: p for p in dir_path.glob("*") if p.is_file() and p.suffix in IMG_EXTS}

def move_pair(stem: str, img_path: Path | None, lbl_path: Path | None, split: str, prefix: str):
    """이미지/라벨 쌍을 prefix__ 붙여 상위 train|val 폴더로 이동"""
    new_stem = f"{prefix}__{stem}"
    if img_path and img_path.exists():
        dst_img = MERGE_ROOT / "images" / split / f"{new_stem}{img_path.suffix.lower()}"
        shutil.move(str(img_path), dst_img)
    if lbl_path and lbl_path.exists():
        dst_lbl = MERGE_ROOT / "labels" / split / f"{new_stem}.txt"
        shutil.move(str(lbl_path), dst_lbl)

def move_csvs_and_classes(src_root: Path):
    """소스 폴더 루트에 있는 csv, classes.txt 정리 이동"""
    prefix = src_root.name
    logs_dir = MERGE_ROOT / "logs"

    # CSV들 이동(+ prefix 붙여 충돌 방지)
    for csv_file in src_root.glob("*.csv"):
        dst = logs_dir / f"{prefix}__{csv_file.name}"
        shutil.move(str(csv_file), dst)

    # classes.txt 백업 이동
    cls = src_root / "classes.txt"
    if cls.exists():
        dst = logs_dir / "classes_sources" / f"{prefix}__classes.txt"
        shutil.move(str(cls), dst)

# ================================================================
# 🚀 병합 로직
# ================================================================
def merge_and_move(src_root: Path):
    """각 good_data 폴더의 train/val 이미지와 라벨을 상위로 이동"""
    prefix = src_root.name
    for split in ["train", "val"]:   # ✅ test 제거
        img_dir = src_root / "images" / split
        lbl_dir = src_root / "labels" / split
        if not img_dir.exists() and not lbl_dir.exists():
            continue

        img_map = map_images(img_dir) if img_dir.exists() else {}
        lbl_map = {p.stem: p for p in lbl_dir.glob("*.txt")} if lbl_dir.exists() else {}

        stems = set(img_map) | set(lbl_map)
        for stem in stems:
            move_pair(stem, img_map.get(stem), lbl_map.get(stem), split, prefix)

    # CSV / classes 정리
    move_csvs_and_classes(src_root)

    # 빈 소스 폴더 제거
    shutil.rmtree(src_root, ignore_errors=True)

# ================================================================
# 🧠 클래스 통합
# ================================================================
def read_and_unify_class_names():
    """logs/classes_sources 아래의 classes.txt들을 비교해 통일. 없으면 기본값."""
    defaults = ["Divot","Fixed_Divot","Diseased_Grass","Confused_Object",
                "Pole","Sprinkler","Drain","Golf ball"]
    src_dir = MERGE_ROOT / "logs" / "classes_sources"
    files = sorted(src_dir.glob("*_classes.txt"))
    if not files:
        return defaults, "default"

    def read_names(p: Path):
        return [ln.strip() for ln in p.read_text(encoding="utf-8").splitlines() if ln.strip()]

    base_names = read_names(files[0])
    consistent = True
    for f in files[1:]:
        if read_names(f) != base_names:
            consistent = False
            break
    if not consistent or not base_names:
        warn = MERGE_ROOT / "logs" / "classes_mismatch.txt"
        with open(warn, "w", encoding="utf-8") as w:
            w.write("WARNING: classes.txt mismatch among sources. Using first one as canonical.\n")
            for f in files:
                w.write(f"- {f.name}\n")
        return (base_names if base_names else defaults), "mismatch"
    return base_names, "ok"

def write_classes_txt(names: list[str]):
    out = MERGE_ROOT / "classes.txt"
    out.write_text("\n".join(names) + "\n", encoding="utf-8")
    print(f"✔ classes.txt -> {out}")

def write_data_yaml(names: list[str]):
    yaml_path = MERGE_ROOT / "data.yaml"
    with open(yaml_path, "w", encoding="utf-8") as f:
        f.write("train: images/train\n")
        f.write("val: images/val\n\n")     # ✅ test 제거
        f.write(f"nc: {len(names)}\n")
        f.write("names: [")
        f.write(", ".join(f"\"{n}\"" for n in names))
        f.write("]\n")
    print(f"✔ data.yaml -> {yaml_path}")

# ================================================================
# 📊 간단 요약
# ================================================================
def quick_summary():
    for split in ["train", "val"]:   # ✅ test 제거
        n_img = len(list((MERGE_ROOT / "images" / split).glob("*")))
        n_lbl = len(list((MERGE_ROOT / "labels" / split).glob("*.txt")))
        print(f"[{split}] images={n_img}, labels={n_lbl}")

# ================================================================
# 🧾 로그 병합
# ================================================================
def merge_all_aug_logs():
    """logs/ 아래 prefix 붙여 옮긴 CSV 중 aug_* 로그를 하나로 합치고 source 컬럼 추가"""
    logs_dir = MERGE_ROOT / "logs"
    out_path = logs_dir / "merged_aug_log.csv"
    candidates = [p for p in logs_dir.glob("*.csv") if "aug" in p.name.lower()]
    if not candidates:
        return
    header = None
    rows = []
    for p in candidates:
        try:
            with open(p, newline="", encoding="utf-8") as f:
                r = csv.reader(f)
                h = next(r, None)
                if h is None:
                    continue
                if header is None:
                    header = h + ["source_csv"]
                elif h != header[:-1]:
                    continue
                for row in r:
                    rows.append(row + [p.name])
        except Exception:
            continue
    if header:
        with open(out_path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(header)
            w.writerows(rows)
        print(f"✔ merged_aug_log.csv -> {out_path}")

# ================================================================
# 🚀 메인 실행
# ================================================================
if __name__ == "__main__":
    ensure_dirs()
    for src in SOURCES:
        if src.exists():
            print(f"→ Moving from: {src}")
            merge_and_move(src)

    names, status = read_and_unify_class_names()
    write_classes_txt(names)
    write_data_yaml(names)

    merge_all_aug_logs()
    quick_summary()
    print("✅ Done. All train/val files MOVED to top-level. CSV/logs & classes organized.")


### 10. 최종 점검

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path

BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

def check_split(split: str):
    print(f"\n=== Checking {split} ===")

    img_dir = BASE / "images" / split
    lbl_dir = BASE / "labels" / split
    if not img_dir.exists() or not lbl_dir.exists():
        print(f"❌ Missing {split} folder")
        return

    imgs = {p.stem for p in img_dir.rglob("*") if p.suffix in IMG_EXTS}
    lbls = {p.stem for p in lbl_dir.rglob("*.txt")}

    only_img = sorted(imgs - lbls)
    only_lbl = sorted(lbls - imgs)

    # ✅ bg 이미지 필터링 (이름에 bg 포함되면 괜찮음)
    only_img_nonbg = [s for s in only_img if "bg" not in s.lower()]

    # 빈 라벨 파일
    empty_lbl = []
    for lbl in lbl_dir.rglob("*.txt"):
        try:
            if lbl.stat().st_size == 0:
                empty_lbl.append(lbl.stem)
        except FileNotFoundError:
            continue

    print(f"Total images: {len(imgs)}, labels: {len(lbls)}")
    print(f"✅ Matched pairs: {len(imgs & lbls)}")

    if only_img_nonbg:
        print(f"⚠️ Images without label (non-bg) ({len(only_img_nonbg)}):")
        for s in only_img_nonbg[:10]:
            print(f"  - {s}")
        if len(only_img_nonbg) > 10:
            print(f"  ... +{len(only_img_nonbg)-10} more")

    if only_lbl:
        print(f"⚠️ Labels without image ({len(only_lbl)}):")
        for s in only_lbl[:10]:
            print(f"  - {s}")
        if len(only_lbl) > 10:
            print(f"  ... +{len(only_lbl)-10} more")

    if empty_lbl:
        print(f"⚠️ Empty label files ({len(empty_lbl)}):")
        for s in empty_lbl[:10]:
            print(f"  - {s}")
        if len(empty_lbl) > 10:
            print(f"  ... +{len(empty_lbl)-10} more")

    # 요약 출력
    if not only_img_nonbg and not only_lbl and not empty_lbl:
        print("✅ All good — image/label pairs match perfectly (bg images allowed).")

if __name__ == "__main__":
    for split in ["train", "val"]:
        check_split(split)



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path

# === YOLO 데이터 루트 경로 설정 ===
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")

# === 점검 대상 폴더 (train/val 둘 다) ===
LABEL_DIRS = [
    BASE / "labels" / "train",
    BASE / "labels" / "val",
]

def remove_empty_labels(label_dirs):
    total_removed = 0
    for lbl_dir in label_dirs:
        if not lbl_dir.exists():
            continue
        empty_files = []
        for lbl in lbl_dir.rglob("*.txt"):
            try:
                if lbl.stat().st_size == 0:  # 파일 크기 0
                    empty_files.append(lbl)
            except FileNotFoundError:
                continue

        if not empty_files:
            print(f"✅ No empty labels in {lbl_dir}")
            continue

        print(f"⚠️ {lbl_dir}: Found {len(empty_files)} empty label files.")
        for lbl in empty_files:
            lbl.unlink()  # 삭제
        print(f"🗑️  Removed {len(empty_files)} empty label files from {lbl_dir}")
        total_removed += len(empty_files)

    print(f"\n=== Done. Total removed: {total_removed} ===")

if __name__ == "__main__":
    remove_empty_labels(LABEL_DIRS)


### 11. 최종데이터 로그 출력

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
from collections import Counter
import csv

# ================================================================
# 🧩 기본 설정
# ================================================================
MERGE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data")
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
SPLITS = ["train", "val"]  # ✅ test 제거

# ================================================================
# 🧠 클래스 이름 로드
# ================================================================
def read_class_names():
    f = MERGE_ROOT / "classes.txt"
    if f.exists():
        names = [ln.strip() for ln in f.read_text(encoding="utf-8").splitlines() if ln.strip()]
        if names:
            return names
    # fallback
    return ["Divot","Fixed_Divot","Diseased_Grass","Confused_Object",
            "Pole","Sprinkler","Drain","Golf ball"]

CLASS_NAMES = read_class_names()
NC = len(CLASS_NAMES)

# ================================================================
# 🧰 유틸
# ================================================================
def list_images(dirpath: Path):
    return [p for p in dirpath.glob("*") if p.suffix in IMG_EXTS and p.is_file()]

def count_split(split: str):
    img_dir = MERGE_ROOT / "images" / split
    lbl_dir = MERGE_ROOT / "labels" / split
    img_count = 0
    labeled_image_stems = set()
    obj_counter = Counter()
    unknown_counter = 0

    if img_dir.exists():
        img_count = len(list_images(img_dir))
    if lbl_dir.exists():
        for txt in lbl_dir.glob("*.txt"):
            stem = txt.stem
            labeled_image_stems.add(stem)
            for line in txt.read_text(encoding="utf-8", errors="ignore").splitlines():
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                try:
                    cid = int(float(parts[0]))
                except:
                    continue
                if 0 <= cid < NC:
                    obj_counter[cid] += 1
                else:
                    unknown_counter += 1

    bg_images = max(0, img_count - len(labeled_image_stems))
    return {
        "images_total": img_count,
        "images_labeled": len(labeled_image_stems),
        "images_bg": bg_images,
        "objs_per_class": obj_counter,
        "objs_unknown": unknown_counter,
    }

# ================================================================
# 📊 메인
# ================================================================
def main():
    # 수집
    results = {s: count_split(s) for s in SPLITS}

    # 전체 합산
    total_imgs = sum(results[s]["images_total"] for s in SPLITS)
    total_objs = Counter()
    for s in SPLITS:
        total_objs.update(results[s]["objs_per_class"])
    grand_objs = sum(total_objs.values())

    # 텍스트 리포트
    out_txt = MERGE_ROOT / "split_object_report_trainval.txt"
    lines = []
    lines.append(f"Root: {MERGE_ROOT}")
    lines.append(f"Classes({NC}): {', '.join(CLASS_NAMES)}\n")

    for s in SPLITS:
        r = results[s]
        lines.append(f"[{s}] images_total={r['images_total']}, "
                     f"labeled={r['images_labeled']}, bg={r['images_bg']}, "
                     f"objs_total={sum(r['objs_per_class'].values())}, unknown={r['objs_unknown']}")
        for i, name in enumerate(CLASS_NAMES):
            lines.append(f"  {i:02d} {name:16s}: {r['objs_per_class'].get(i,0)}")

        split_obj_sum = max(1, sum(r["objs_per_class"].values()))
        ratios = ", ".join([f"{name}:{r['objs_per_class'].get(i,0)/split_obj_sum:.3f}"
                            for i, name in enumerate(CLASS_NAMES)])
        lines.append(f"  per-class ratios: {ratios}\n")

    # 전체 요약
    lines.append("=== OVERALL (train/val only) ===")
    lines.append(f"Total images: {total_imgs}")
    lines.append(f"Total objects: {grand_objs}")
    lines.append("Objects per class (overall):")
    for i, name in enumerate(CLASS_NAMES):
        lines.append(f"  {i:02d} {name:16s}: {total_objs.get(i,0)}")
    if grand_objs > 0:
        ratios_overall = ", ".join([f"{name}:{total_objs.get(i,0)/grand_objs:.3f}"
                                    for i, name in enumerate(CLASS_NAMES)])
        lines.append(f"Overall ratios: {ratios_overall}")

    out_txt.write_text("\n".join(lines), encoding="utf-8")

    # CSV 저장
    out_csv = MERGE_ROOT / "split_object_report_trainval.csv"
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        header = ["split","images_total","images_labeled","images_bg","objs_total"] + \
                 [f"obj_{i}_{CLASS_NAMES[i]}" for i in range(NC)]
        w.writerow(header)
        for s in SPLITS:
            r = results[s]
            row = [s, r["images_total"], r["images_labeled"], r["images_bg"],
                   sum(r["objs_per_class"].values())]
            row += [r["objs_per_class"].get(i,0) for i in range(NC)]
            w.writerow(row)
        # overall
        row = ["overall", total_imgs, sum(results[s]["images_labeled"] for s in SPLITS),
               sum(results[s]["images_bg"] for s in SPLITS), grand_objs]
        row += [total_objs.get(i,0) for i in range(NC)]
        w.writerow(row)

    # 콘솔 요약
    print(f"✅ Saved: {out_txt}")
    print(f"✅ Saved: {out_csv}")
    for s in SPLITS:
        r = results[s]
        print(f"[{s}] images={r['images_total']}  labeled={r['images_labeled']}  "
              f"bg={r['images_bg']}  objs={sum(r['objs_per_class'].values())}")

    print("\n=== TOTAL (train/val only) ===")
    print(f"images={total_imgs}  objects={grand_objs}")

# ================================================================
# 🚀 실행
# ================================================================
if __name__ == "__main__":
    main()


### 11. 학습

In [ ]:
from ultralytics import YOLO
from pathlib import Path
from datetime import datetime
from ultralytics.utils.torch_utils import strip_optimizer
import csv, math
import shutil   # best_map50 저장용

# ==============================
# 실험 관련 기본 설정
# ==============================

# 데이터 설정: train/val 경로, 클래스 수/이름이 정확해야 함
#DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251014_merge_data_only_new_camera/data.yaml"
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251017_merge_data_add/data.yaml"
# 모델 선택: s(가볍고 빠름) → 결과 좋아지면 m으로 확장
MODEL_NAME = "yolov8s.pt"

# ====== 사용자 지정 하이퍼파라미터 ======
IMG_SIZE  = 640        # 입력 해상도(정사각). 소물체 ↑. OOM 시 896/768로   
OPTIM     = "SGD"      # Optimizer 선택 ("SGD","AdamW","Adam" 등)
CLS_W     = 0.5        # 클래스 분류 손실 가중(기본≈0.5)
BOX_W     = 7.5       # bbox 회귀 손실 가중(기본≈7.5)|
DFL_W     = 1.5       # Distribution Focal Loss 가중(기본≈1.5)
EPOCHS    = 500          # 필요시 조정
DATA_TAG  = "20251017" # 데이터셋 날짜(데이터셋에 맞게 수정)
RECT_MODE = False      # ✅ rect 설정(True / False) → 폴더명에도 기록됨 디폴트는 False
SEED = 33


# 오늘 날짜 자동 삽입
TODAY     = datetime.now().strftime("%Y%m%d")   # 예: 20250921

# ==============================
# 모델 이름 자동 추출
# ==============================
MODEL_TAG = Path(MODEL_NAME).stem.lower()  # ex) yolov8s.pt → yolov8s
print(f"선택한 모델 태그: {MODEL_TAG}")

# ==============================
# 실행 옵션
# ==============================
SAVE_PERIOD   = 1          # epoch*.pt 저장 주기(1이면 매 에포크 저장)
STRIP_COPIES  = True       # 복사본을 추론용으로 슬림화(optimizer 등 제거)
CLEAN_EPOCHS  = True       # 복사/요약 후 남은 epoch*.pt 삭제(용량 절감)

# ==============================
# 출력 폴더 이름 자동 생성
#    예: 20250919_yolov8s_img896_SGD_cls0.5_box7.5_dfl1.5_rectFalse_20250921
# ==============================
exp_name = (
    f"{DATA_TAG}_data_{MODEL_TAG}_img{IMG_SIZE}_{OPTIM}"
    f"_cls{CLS_W}_box{BOX_W}_dfl{DFL_W}_rect{str(RECT_MODE)}_seed_{SEED}_{TODAY}_add"
)
print("실험 폴더 이름:", exp_name)


# ==============================
# 모델 학습 시작
# ==============================
model = YOLO(MODEL_NAME)  # ← 여기만 바뀌면 됩니다

results = model.train(
    # =========================
    # 데이터 & 하드웨어 설정
    # =========================
    data=DATA_YAML,          # (str) 학습/검증/테스트 경로와 클래스 정보를 담은 data.yaml
    imgsz=IMG_SIZE,          # (int) 입력 이미지를 정사각형으로 resize할 크기 (ex: 640)
    batch=-1,                # (int|float) 배치 크기: -1 → GPU 메모리 기준 자동 탐색
    device=0,                # (int|str|list) GPU 인덱스/리스트 또는 'cpu'/'mps'
    workers=8,               # (int) DataLoader 병렬 로드 worker 수 (CPU 코어 상황에 맞춰)

    # =========================
    # 학습 스케줄/최적화
    # =========================
    epochs=EPOCHS,              # (int) 총 학습 epoch 수
    patience=50,             # (int) val metric 개선 없을 때 조기종료 대기 epoch
    lr0=0.01,                # (float) 초기 학습률 (SGD는 0.01 권장, Adam 계열은 더 낮게) 0.01 0.001
    lrf=0.01,                # (float) 최종 학습률 비율 (lr0 * lrf = 마지막 epoch 학습률)
    momentum=0.937,          # (float) SGD 모멘텀 또는 Adam β1 값   
    weight_decay=0.0005,     # (float) L2 정규화 계수 (과적합 방지)   adame0.01  sgd 0.0005

    optimizer=OPTIM,         # (str) 최적화 알고리즘: 'SGD','Adam','AdamW' 등
    cos_lr=False,            # (bool|int) 코사인 학습률 스케줄 사용 여부 (0 → 비활성)
    warmup_epochs=0,         # (float) 워밍업 epoch 수 (0 → 워밍업 없음)
    warmup_momentum=0,       # (float) 워밍업 단계에서 사용할 초기 모멘텀
    warmup_bias_lr=0,        # (float) 워밍업 단계에서 bias 파라미터 학습률

    # =========================
    # 손실 가중치
    # =========================
    box=BOX_W,                 # (float) 바운딩 박스 회귀 손실 가중치
    cls=CLS_W,                 # (float) 클래스 분류 손실 가중치
    dfl=DFL_W,                 # (float) Distribution Focal Loss 가중치 (YOLOv8 box refinement)

    # =========================
    # 데이터 증강(Online Augmentation)
    # =========================
    mosaic=0.0,              # (float) mosaic 증강 비율 (0 → 사용 안함)
    close_mosaic=0,          # (int) 마지막 N epoch 동안 mosaic 비활성 (0 → 비활성화)
    mixup=0.0,               # (float) mixup 비율
    copy_paste=0.0,          # (float) 객체 copy-paste 비율
    hsv_h=0.0,             # (float) Hue 색조 변동 범위
    hsv_s=0.0,               # (float) 채도(Saturation) 변동 범위
    hsv_v=0.0,               # (float) 밝기(Value) 변동 범위
    degrees=0.0,             # (float) 회전 각도 범위
    translate=0.0,           # (float) 이미지 평행이동 비율
    scale=0.0,               # (float) 스케일 변형 비율
    shear=0.0,               # (float) 기울이기 변형 비율
    perspective=0,           # (float) 원근 변형 비율
    erasing=0.0,             # (float) 랜덤 영역 삭제 비율

    # =========================
    # 고급 옵션 및 재현성
    # =========================
    rect=RECT_MODE,          # (bool) 배치 단위로 aspect ratio 유지 여부 (train 기본 False)
    cache="disk",            # (bool|'ram'|'disk') 이미지/라벨 캐시: 'disk'는 I/O 절감
    multi_scale=False,       # (bool) 다중 스케일 학습 활성화 (0 → 비활성)
    single_cls=False,        # (bool) 모든 클래스를 하나로 통합
    classes=None,            # (list[int]|int) 특정 클래스만 학습. 0 → 전체 클래스 사용
    fraction=1.0,            # (float) 데이터셋 사용 비율 (1.0 → 전체 데이터)
    seed=SEED,               # (int) 랜덤 시드 (재현성)
    deterministic=True,      # (bool) 완전 결정론적 알고리즘 사용
    amp=True,                # (bool) 자동 혼합정밀도(fp16) 학습
    freeze=0,                # (int|list) 처음 N개 레이어 혹은 특정 레이어 인덱스 고정
    compile=False,           # (bool|str) torch.compile 사용 (0 → 비활성)
    profile=False,           # (bool) ONNX/TensorRT 프로파일링
    val=True,                # (bool) 학습 중 검증 실행
    save=True,               # (bool) 체크포인트 및 최종 가중치 저장
    save_period=SAVE_PERIOD, # (int) 몇 epoch마다 중간 가중치 저장 (0 → 비활성)

    # =========================
    # 결과 저장
    # =========================
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # (str) 상위 프로젝트 폴더
    name=exp_name,           # (str) 프로젝트 하위 폴더명
    exist_ok=True            # (bool) 동일 이름 폴더가 있어도 덮어쓰기 허용
)


# 🔹 sum4 가중치 (사용자가 직접 수정 가능)
WEIGHTS_SUM4 = {
    "precision": 0.20,
    "recall":    0.30,
    "map50":     0.25,
    "map5095":   0.25,
}


# ==============================
# 후처리: results.csv 기반 베스트 체크포인트 정리
# ==============================
proj_dir    = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo")
exp_dir     = proj_dir / exp_name
weights_dir = exp_dir / "weights"
csv_path    = exp_dir / "results.csv"

if not csv_path.exists():
    raise FileNotFoundError(f"results.csv을 찾을 수 없습니다: {csv_path}")

CANDIDATES = {
    "precision": ["metrics/precision(B)", "metrics/precision", "precision"],
    "recall":    ["metrics/recall(B)",    "metrics/recall",    "recall"],
    "map50":     ["metrics/mAP50(B)",     "metrics/mAP50",     "map50"],
    "map5095":   ["metrics/mAP50-95(B)",  "metrics/mAP50-95",  "mAP50-95"],
}

def read_results_csv(path: Path):
    with path.open("r", newline="") as f:
        reader = csv.DictReader(f)
        rows = list(reader)
        fields = reader.fieldnames or []
    return rows, fields

def pick_col(cands, fieldnames):
    for c in cands:
        if c in fieldnames: return c
    raise KeyError(f"CSV에 해당 메트릭 컬럼이 없음: {cands}")

def to_float_safe(v):
    try:
        x = float(v); return x if not math.isnan(x) else -1.0
    except: return -1.0

rows, fields = read_results_csv(csv_path)
col_prec    = pick_col(CANDIDATES["precision"], fields)
col_rec     = pick_col(CANDIDATES["recall"],    fields)
col_map50   = pick_col(CANDIDATES["map50"],     fields)
col_map5095 = pick_col(CANDIDATES["map5095"],   fields)

# ==============================
# Top3 관리 함수
# ==============================
def update_top3(top3, value, epoch):
    b_val, b_ep = top3[0]; s_val, s_ep = top3[1]; t_val, t_ep = top3[2]
    if value > b_val or (value == b_val and (b_ep == -1 or epoch < b_ep)):
        if b_ep != -1 and b_ep != epoch:
            t_val, t_ep = s_val, s_ep
            s_val, s_ep = b_val, b_ep
        b_val, b_ep = value, epoch
    elif epoch != b_ep and (value > s_val or (value == s_val and (s_ep == -1 or epoch < s_ep))):
        if s_ep != -1 and s_ep != epoch:
            t_val, t_ep = s_val, s_ep
        s_val, s_ep = value, epoch
    elif epoch not in (b_ep, s_ep) and (value > t_val or (value == t_val and (t_ep == -1 or epoch < t_ep))):
        t_val, t_ep = value, epoch
    top3[0], top3[1], top3[2] = (b_val, b_ep), (s_val, s_ep), (t_val, t_ep)

top3 = {m:[(-1.0,-1)]*3 for m in ["precision","recall","map50","map5095","sum4"]}

for r in rows:
    if not r.get("epoch"): continue
    e_csv = int(r["epoch"])
    p,rc,m50,m95 = map(to_float_safe, [r.get(col_prec), r.get(col_rec), r.get(col_map50), r.get(col_map5095)])
    update_top3(top3["precision"], p, e_csv)
    update_top3(top3["recall"],    rc, e_csv)
    update_top3(top3["map50"],     m50, e_csv)
    update_top3(top3["map5095"],   m95, e_csv)
    sum4 = (p*WEIGHTS_SUM4["precision"] +
            rc*WEIGHTS_SUM4["recall"] +
            m50*WEIGHTS_SUM4["map50"] +
            m95*WEIGHTS_SUM4["map5095"])
    update_top3(top3["sum4"], sum4, e_csv)

# ==============================
# 체크포인트 복사
# ==============================
best_file, last_file = weights_dir/"best.pt", weights_dir/"last.pt"
kept_files = set([f.name for f in (best_file,last_file) if f.exists()])
summary_lines=[]

def ckpt_path_from_csv_epoch(e_csv: int) -> Path:
    return weights_dir / f"epoch{e_csv-1}.pt"

def same_file(a: Path, b: Path) -> bool:
    return a.exists() and b.exists() and a.read_bytes() == b.read_bytes()

def maybe_copy(tag_prefix, rank, epoch_csv, value):
    if epoch_csv < 0 or value < 0: return
    src = ckpt_path_from_csv_epoch(epoch_csv)
    if not src.exists(): return
    if same_file(src,best_file) or same_file(src,last_file): return
    dst = weights_dir / f"{tag_prefix}_{rank}_epoch{epoch_csv}_{value:.5f}.pt"
    shutil.copy2(src,dst)
    if STRIP_COPIES: strip_optimizer(str(dst))
    kept_files.add(dst.name)
    summary_lines.append(f"{tag_prefix} {rank}: epoch={epoch_csv}, value={value:.6f} -> {dst.name}")

for metric in ["precision","recall","map50","map5095","sum4"]:
    for idx,rank in enumerate(["BEST","SECOND","THIRD"]):
        v,e = top3[metric][idx]
        maybe_copy(metric,rank,e,v)

# ==============================
# 남은 epoch 삭제
# ==============================
if CLEAN_EPOCHS:
    for p in weights_dir.glob("epoch*.pt"):
        if p.name not in kept_files:
            p.unlink(missing_ok=True)

summary_path = exp_dir / "best_by_metric_summary.txt"
with summary_path.open("w") as f:
    f.write("=== Top3 by Metric (with weighted sum4) ===\n")
    f.write(f"WEIGHTS_SUM4 = {WEIGHTS_SUM4}\n\n")
    for line in summary_lines: f.write(line+"\n")

print("\n[완료] 보존된 체크포인트:")
for n in sorted(kept_files): print(" -",n)
print(f"요약 저장됨: {summary_path}")


### 여러시드 연속적으로 이어서

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from ultralytics import YOLO
from pathlib import Path
from datetime import datetime
from ultralytics.utils.torch_utils import strip_optimizer
import csv, math, shutil, gc, time

# ==============================
# 실험 관련 기본 설정
# ==============================
DATA_YAML  = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251107_merge_data/data.yaml"
MODEL_NAME = "yolov8m.pt"

# ====== 사용자 지정 하이퍼파라미터 ======
IMG_SIZE  = 640
OPTIM     = "SGD"
CLS_W     = 0.5
BOX_W     = 7.5
DFL_W     = 1.5
EPOCHS    = 1000
DATA_TAG  = "20251107"
RECT_MODE = False
SEEDS     = [25, 33, 57]   # ✅ 여기만 바꾸면 씨드 자동 순차 실행

# 오늘 날짜
TODAY     = datetime.now().strftime("%Y%m%d")

# ==============================
# 실행 옵션
# ==============================
SAVE_PERIOD   = 1
STRIP_COPIES  = True
CLEAN_EPOCHS  = True

# 결과 저장 기본 경로
PROJECT_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo")

# 🔹 sum4 가중치 (사용자가 직접 수정 가능)
WEIGHTS_SUM4 = {
    "precision": 0.20,
    "recall":    0.30,
    "map50":     0.25,
    "map5095":   0.25,
}

# ==============================
# CSV 유틸
# ==============================
CANDIDATES = {
    "precision": ["metrics/precision(B)", "metrics/precision", "precision"],
    "recall":    ["metrics/recall(B)",    "metrics/recall",    "recall"],
    "map50":     ["metrics/mAP50(B)",     "metrics/mAP50",     "map50"],
    "map5095":   ["metrics/mAP50-95(B)",  "metrics/mAP50-95",  "mAP50-95"],
}

def read_results_csv(path: Path):
    with path.open("r", newline="") as f:
        reader = csv.DictReader(f)
        rows = list(reader)
        fields = reader.fieldnames or []
    return rows, fields

def pick_col(cands, fieldnames):
    for c in cands:
        if c in fieldnames:
            return c
    raise KeyError(f"CSV에 해당 메트릭 컬럼이 없음: {cands}")

def to_float_safe(v):
    try:
        x = float(v)
        return x if not math.isnan(x) else -1.0
    except:
        return -1.0

def update_top3(top3, value, epoch):
    b_val, b_ep = top3[0]; s_val, s_ep = top3[1]; t_val, t_ep = top3[2]
    if value > b_val or (value == b_val and (b_ep == -1 or epoch < b_ep)):
        if b_ep != -1 and b_ep != epoch:
            t_val, t_ep = s_val, s_ep
            s_val, s_ep = b_val, b_ep
        b_val, b_ep = value, epoch
    elif epoch != b_ep and (value > s_val or (value == s_val and (s_ep == -1 or epoch < s_ep))):
        if s_ep != -1 and s_ep != epoch:
            t_val, t_ep = s_val, s_ep
        s_val, s_ep = value, epoch
    elif epoch not in (b_ep, s_ep) and (value > t_val or (value == t_val and (t_ep == -1 or epoch < t_ep))):
        t_val, t_ep = value, epoch
    top3[0], top3[1], top3[2] = (b_val, b_ep), (s_val, s_ep), (t_val, t_ep)

def process_one_run(exp_dir: Path, weights_dir: Path):
    """results.csv를 읽어 Top3 체크포인트를 보존하고 나머지 epoch*.pt 정리"""
    csv_path = exp_dir / "results.csv"
    if not csv_path.exists():
        raise FileNotFoundError(f"results.csv을 찾을 수 없습니다: {csv_path}")

    rows, fields = read_results_csv(csv_path)
    col_prec    = pick_col(CANDIDATES["precision"], fields)
    col_rec     = pick_col(CANDIDATES["recall"],    fields)
    col_map50   = pick_col(CANDIDATES["map50"],     fields)
    col_map5095 = pick_col(CANDIDATES["map5095"],   fields)

    top3 = {m:[(-1.0,-1)]*3 for m in ["precision","recall","map50","map5095","sum4"]}
    for r in rows:
        if not r.get("epoch"):
            continue
        e_csv = int(r["epoch"])
        p, rc, m50, m95 = map(to_float_safe, [r.get(col_prec), r.get(col_rec), r.get(col_map50), r.get(col_map5095)])
        update_top3(top3["precision"], p, e_csv)
        update_top3(top3["recall"],    rc, e_csv)
        update_top3(top3["map50"],     m50, e_csv)
        update_top3(top3["map5095"],   m95, e_csv)
        sum4 = (p*WEIGHTS_SUM4["precision"] +
                rc*WEIGHTS_SUM4["recall"] +
                m50*WEIGHTS_SUM4["map50"] +
                m95*WEIGHTS_SUM4["map5095"])
        update_top3(top3["sum4"], sum4, e_csv)

    best_file, last_file = weights_dir/"best.pt", weights_dir/"last.pt"
    kept_files = set([f.name for f in (best_file, last_file) if f.exists()])
    summary_lines=[]

    def ckpt_path_from_csv_epoch(e_csv: int) -> Path:
        return weights_dir / f"epoch{e_csv-1}.pt"

    def same_file(a: Path, b: Path) -> bool:
        return a.exists() and b.exists() and a.read_bytes() == b.read_bytes()

    def maybe_copy(tag_prefix, rank, epoch_csv, value):
        if epoch_csv < 0 or value < 0:
            return
        src = ckpt_path_from_csv_epoch(epoch_csv)
        if not src.exists():
            return
        if same_file(src, best_file) or same_file(src, last_file):
            return
        dst = weights_dir / f"{tag_prefix}_{rank}_epoch{epoch_csv}_{value:.5f}.pt"
        shutil.copy2(src, dst)
        if STRIP_COPIES:
            strip_optimizer(str(dst))
        kept_files.add(dst.name)
        summary_lines.append(f"{tag_prefix} {rank}: epoch={epoch_csv}, value={value:.6f} -> {dst.name}")

    for metric in ["precision","recall","map50","map5095","sum4"]:
        for idx, rank in enumerate(["BEST","SECOND","THIRD"]):
            v, e = top3[metric][idx]
            maybe_copy(metric, rank, e, v)

    # 남은 epoch 삭제
    if CLEAN_EPOCHS:
        for p in weights_dir.glob("epoch*.pt"):
            if p.name not in kept_files:
                p.unlink(missing_ok=True)

    summary_path = exp_dir / "best_by_metric_summary.txt"
    with summary_path.open("w") as f:
        f.write("=== Top3 by Metric (with weighted sum4) ===\n")
        f.write(f"WEIGHTS_SUM4 = {WEIGHTS_SUM4}\n\n")
        for line in summary_lines:
            f.write(line + "\n")

    print("\n[완료] 보존된 체크포인트:")
    for n in sorted(kept_files):
        print(" -", n)
    print(f"요약 저장됨: {summary_path}")


# ==============================
# 씨드별 순차 실행 + 쿨다운
# ==============================
MODEL_TAG = Path(MODEL_NAME).stem.lower()
print(f"선택한 모델 태그: {MODEL_TAG}")

for seed in SEEDS:
    exp_name = (
        f"{DATA_TAG}_data_{MODEL_TAG}_img{IMG_SIZE}_{OPTIM}"
        f"_cls{CLS_W}_box{BOX_W}_dfl{DFL_W}_rect{str(RECT_MODE)}_seed_{seed}_{TODAY}"
    )
    print("\n" + "="*80)
    print(f"[SEED {seed}] 실험 폴더 이름: {exp_name}")
    print("="*80)

    model = YOLO(MODEL_NAME)

    _ = model.train(
        data=DATA_YAML,
        imgsz=IMG_SIZE,
        batch=-1,
        device=0,
        workers=8,
        epochs=EPOCHS,
        patience=100,
        lr0=0.01,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        optimizer=OPTIM,
        cos_lr=False,
        warmup_epochs=0,
        warmup_momentum=0,
        warmup_bias_lr=0,
        box=BOX_W,
        cls=CLS_W,
        dfl=DFL_W,
        mosaic=0.0,
        mixup=0.0,
        copy_paste=0.0,
        hsv_h=0.0,
        hsv_s=0.0,
        hsv_v=0.0,
        degrees=0.0,
        translate=0.0,
        scale=0.0,
        shear=0.0,
        perspective=0,
        erasing=0.0,
        rect=RECT_MODE,
        cache="disk",
        multi_scale=False,
        single_cls=False,
        seed=seed,
        deterministic=True,
        amp=True,
        freeze=0,
        compile=False,
        profile=False,
        val=True,
        save=True,
        save_period=SAVE_PERIOD,
        project=str(PROJECT_DIR),
        name=exp_name,
        exist_ok=True
    )

    # 결과 정리 (Top3 복사/슬림화/epoch 정리)
    exp_dir     = PROJECT_DIR / exp_name
    weights_dir = exp_dir / "weights"
    process_one_run(exp_dir, weights_dir)

    # 메모리 정리
    del model
    gc.collect()
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    except Exception:
        pass

    # 🔹 5분 쿨다운
    COOLDOWN_TIME = 120  # 초 단위 (2분)
    print(f"\n[SEED {seed}] 학습 완료 — 다음 시드 전 {COOLDOWN_TIME//60}분간 휴식 중...\n")
    for i in range(COOLDOWN_TIME, 0, -30):
        print(f"  ⏳ 남은 대기 시간: {i}초", end="\r")
        time.sleep(30)
    print("\n")

print("\n✅ 모든 씨드 학습 및 후처리가 완료되었습니다.")



### 11. 결과확인 (weight)

In [15]:
# ==============================
# YOLOv8 Test Metrics with TensorRT Weight
# ==============================
from ultralytics import YOLO
from pathlib import Path

# TensorRT 엔진 & 데이터 설정
WEIGHT    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls1.0_box7.5_dfl1.5_rectFalse_seed_25_20251123/weights/recall_BEST_epoch23_0.77358.pt"
#WEIGHT    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls1.0_box7.5_dfl1.5_rectFalse_seed_25_20251123/weights/best.pt"
#WEIGHT    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_25_20251122_2/weights/sum4_BEST_epoch36_0.74756.pt"
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/20251113_test_data_add_leaves/data.yaml"

# .engine 파일 직접 로드
model = YOLO(WEIGHT)

#--- Test split 평가 ---
results = model.val(
    # ===== 데이터 & 디바이스 =====
    data=DATA_YAML,    # data.yaml (train/val/test 경로 포함)
    split="test",      # "val" | "test" | "train" 선택
    imgsz=640,         # (int) 입력 해상도 (default=640)
    batch=1,           # (int) 배치 크기 (default=16)
    rect=False,        # (bool) aspect ratio 유지 여부 (default=True)
    device=0,          # (int|str|list) 디바이스 (default=None → 자동 선택)
    deterministic=True,  # ✅ 모든 연산 고정 (가능한 경우)
    seed=0,     
    # ===== NMS / 임계값 =====
    conf=0.5,        # (float) confidence threshold (default=0.001)
    iou=0.5,          # (float) NMS IoU threshold (default=0.6)
)

print(results)

Ultralytics 8.3.202 🚀 Python-3.10.12 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,128,680 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 9222.9±795.3 MB/s, size: 1736.1 KB)
val: Scanning /home/dw/ws_job_msislab/Golf_Project/data/for_test/20251113_test_data_add_leaves/labels/test.cache... 1237 images, 116 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1349/1349 3.9Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1349/1349 169.5it/s 8.0s<0.0s
                   all       1349       4192      0.888      0.813      0.869      0.527
                 Divot       1022       1416       0.89      0.823      0.879      0.525
           Fixed_Divot        883       2535      0.871      0.811      0.865      0.534
        Diseased_Grass         95        123      0.754      0.398      0.565      0.326
                  Pole         22    